In [1]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

## Type 4 

In [ ]:
import re
import numpy as np
import pandas as pd
from openai import OpenAI

client = OpenAI(
    api_key="sk-proj-"
)

In [3]:
data = pd.read_excel('/kaggle/input/confer-extension/Raw_Dataset/Type4_possessive.xlsx')
data

,premise,hypothesis,gold_label,trigger,SetID,UID
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,1,type4_possessive_01
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,3,type4_possessive_03
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,4,type4_possessive_04
...,...,...,...,...,...,...
94,"If Christina is a makeup artist, she'll use he...",Christina has a makeup sponge.,E,possessive,94,type4_possessive_94
95,"If Rahim is a clergyman, he'll carry around hi...",Rahim has a holy book.,E,possessive,95,type4_possessive_95
96,"If James is a police officer, he'll leave his ...",James has a gun.,E,possessive,96,type4_possessive_96
97,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97


In [8]:
def extract_type4_trigger_words(original_sentence):

    prompt = (
        f'''Extract the possessive phrase from the following sentence.

        For example: 
        Input: If Randolf is a carpenter, he'll use his beading tools for designing.
        Output: his beading tools

        Input: If Liam is an aerospace engineer, he'll test the designs in his wind tunnel.
        Output: his wind tunnel

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [4]:
def generate_type4_conjunction(original_sentence):

    prompt = (
        f'''Modify the following sentence to be in the form:
        If [subject] is an [occupation] and he [verb] his [tool], it's okay. 

        For example: 
        Input: 'If Randolf is a carpenter, he'll use his beading tools for designing.'
        Output: 'If Randolf is a carpenter and he uses his beading tools for designing, it's okay.'

        Input: 'If Liam is an aerospace engineer, he'll test the designs in his wind tunnel.'
        Output: 'If Liam is an aerospace engineer and he tests the designs in his wind tunnel, it's okay.' 

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [4]:
original_sentence = data['premise'].iloc[1]
original_sentence

"If Bill is a content strategist, he'll manage everything in his content management application."

In [7]:
sentence = generate_type4_conjunction(original_sentence) 
sentence

"'If Bill is a content strategist and he manages everything in his content management application, it's okay.'"

In [5]:
data_sample = data[2:10]
data_sample

,premise,hypothesis,gold_label,trigger,SetID,UID
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,3,type4_possessive_03
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,4,type4_possessive_04
5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,5,type4_possessive_05
6,"If Edward is a playwright, he'll bring his pen.",Edward has a pen.,E,possessive,6,type4_possessive_06
7,"If William is a notary, he'll bring his notary...",William has a notary seal.,E,possessive,7,type4_possessive_07
8,"If Charles is an urban forester, he'll prune t...",Charles has arborist tools.,E,possessive,8,type4_possessive_08
9,"If Clyde is a paleontologist, he'll bring his ...",Clyde has field excavation tools.,E,possessive,9,type4_possessive_09


In [13]:
data_sample['Conjunction Premise'] = data_sample['premise'].apply(generate_type4_conjunction)
data_sample

/tmp/ipykernel_35/561628950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Conjunction Premise'] = data_sample['premise'].apply(generate_type4_conjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,If Steve is a conservation officer and he sets...
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,If Matt is a scuba diver and he brings his wet...
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,'If Noah is a pastry chef and he pipes frostin...
5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,5,type4_possessive_05,If Donald is a crime scene investigator and he...
6,"If Edward is a playwright, he'll bring his pen.",Edward has a pen.,E,possessive,6,type4_possessive_06,If Edward is a playwright and he brings his pe...
7,"If William is a notary, he'll bring his notary...",William has a notary seal.,E,possessive,7,type4_possessive_07,If William is a notary and he brings his notar...
8,"If Charles is an urban forester, he'll prune t...",Charles has arborist tools.,E,possessive,8,type4_possessive_08,If Charles is an urban forester and he prunes ...
9,"If Clyde is a paleontologist, he'll bring his ...",Clyde has field excavation tools.,E,possessive,9,type4_possessive_09,If Clyde is a paleontologist and he brings his...


In [14]:
pd.set_option('display.max_colwidth', None)
data_sample

,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise
2,"If Steve is a conservation officer, he'll set up his wildlife trap to capture animals for research or relocation.",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,"If Steve is a conservation officer and he sets up his wildlife trap to capture animals for research or relocation, it's okay."
3,"If Matt is a scuba diver, he'll bring his wetsuit.",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,"If Matt is a scuba diver and he brings his wetsuit, it's okay."
4,"If Noah is a pastry chef, he'll pipe frosting with his pastry bag.",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,"'If Noah is a pastry chef and he pipes frosting with his pastry bag, it's okay.'"
5,"If Donald is a crime scene investigator, he'll use his evidence bags to collect and secure crucial evidence.",Donald has evidence bags.,E,possessive,5,type4_possessive_05,"If Donald is a crime scene investigator and he uses his evidence bags to collect and secure crucial evidence, it's okay."
6,"If Edward is a playwright, he'll bring his pen.",Edward has a pen.,E,possessive,6,type4_possessive_06,"If Edward is a playwright and he brings his pen, it's okay."
7,"If William is a notary, he'll bring his notary seal.",William has a notary seal.,E,possessive,7,type4_possessive_07,"If William is a notary and he brings his notary seal, it's okay."
8,"If Charles is an urban forester, he'll prune trees with his arborist tools.",Charles has arborist tools.,E,possessive,8,type4_possessive_08,"If Charles is an urban forester and he prunes trees with his arborist tools, it's okay."
9,"If Clyde is a paleontologist, he'll bring his field excavation tools with him.",Clyde has field excavation tools.,E,possessive,9,type4_possessive_09,"If Clyde is a paleontologist and he brings his field excavation tools with him, it's okay."


In [15]:
data_sample.to_csv('type4_ex2-10.csv')

In [6]:
remaining_data = data[10:]
remaining_data

,premise,hypothesis,gold_label,trigger,SetID,UID
10,"If Joseph is a mountain guide, he'll bring his...",Joseph has a climbing harness.,E,possessive,10,type4_possessive_10
11,"If Theodore is a ski teacher, he'll bring his ...",Theodore has skis.,E,possessive,11,type4_possessive_11
12,"If David is an actor, he'll memorize his scripts.",David has scripts.,E,possessive,12,type4_possessive_12
13,"If Andrew is a home inspector, he'll use his m...",Andrew has a moisture meter.,E,possessive,13,type4_possessive_13
14,"If James is an athletic trainer, he'll bring h...",James has a first aid kit.,E,possessive,14,type4_possessive_14
...,...,...,...,...,...,...
94,"If Christina is a makeup artist, she'll use he...",Christina has a makeup sponge.,E,possessive,94,type4_possessive_94
95,"If Rahim is a clergyman, he'll carry around hi...",Rahim has a holy book.,E,possessive,95,type4_possessive_95
96,"If James is a police officer, he'll leave his ...",James has a gun.,E,possessive,96,type4_possessive_96
97,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97


In [17]:
remaining_data['Conjunction Premise'] = remaining_data['premise'].apply(generate_type4_conjunction)
remaining_data

/tmp/ipykernel_35/1153855410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Conjunction Premise'] = remaining_data['premise'].apply(generate_type4_conjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise
10,"If Joseph is a mountain guide, he'll bring his climbing harness.",Joseph has a climbing harness.,E,possessive,10,type4_possessive_10,"If Joseph is a mountain guide and he brings his climbing harness, it's okay."
11,"If Theodore is a ski teacher, he'll bring his skis.",Theodore has skis.,E,possessive,11,type4_possessive_11,"If Theodore is a ski teacher and he brings his skis, it's okay."
12,"If David is an actor, he'll memorize his scripts.",David has scripts.,E,possessive,12,type4_possessive_12,"If David is an actor and he memorizes his scripts, it's okay."
13,"If Andrew is a home inspector, he'll use his moisture meter to detect water damage.",Andrew has a moisture meter.,E,possessive,13,type4_possessive_13,"If Andrew is a home inspector and he uses his moisture meter to detect water damage, it's okay."
14,"If James is an athletic trainer, he'll bring his first aid kit.",James has a first aid kit.,E,possessive,14,type4_possessive_14,"If James is an athletic trainer and he brings his first aid kit, it's okay."
...,...,...,...,...,...,...,...
94,"If Christina is a makeup artist, she'll use her makeup sponge to fix Sara's foundation.",Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,"If Christina is a makeup artist and she uses her makeup sponge to fix Sara's foundation, it's okay."
95,"If Rahim is a clergyman, he'll carry around his holy book everywhere.",Rahim has a holy book.,E,possessive,95,type4_possessive_95,"'If Rahim is a clergyman and he carries around his holy book everywhere, it's okay.'"
96,"If James is a police officer, he'll leave his gun home.",James has a gun.,E,possessive,96,type4_possessive_96,"If James is a police officer and he leaves his gun at home, it's okay."
97,"If Maya is a piano teacher, she'll bring her metronome to school.",Maya has a metronome.,E,possessive,97,type4_possessive_97,"If Maya is a piano teacher and she brings her metronome to school, it's okay."


In [18]:
remaining_data.to_csv('type4.csv')

## Extracting Trigger Words

In [10]:
extract_type4_trigger_words(original_sentence) 

'his content management application'

In [11]:
data_sample['Trigger Words'] = data_sample['premise'].apply(extract_type4_trigger_words)
data_sample

/tmp/ipykernel_35/3344803349.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Trigger Words'] = data_sample['premise'].apply(extract_type4_trigger_words)


,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,his wildlife trap
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,his wetsuit
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,his pastry bag
5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,5,type4_possessive_05,his evidence bags
6,"If Edward is a playwright, he'll bring his pen.",Edward has a pen.,E,possessive,6,type4_possessive_06,his pen
7,"If William is a notary, he'll bring his notary...",William has a notary seal.,E,possessive,7,type4_possessive_07,his notary seal
8,"If Charles is an urban forester, he'll prune t...",Charles has arborist tools.,E,possessive,8,type4_possessive_08,his arborist tools
9,"If Clyde is a paleontologist, he'll bring his ...",Clyde has field excavation tools.,E,possessive,9,type4_possessive_09,his field excavation tools


In [12]:
remaining_data['Trigger Words'] = remaining_data['premise'].apply(extract_type4_trigger_words)
remaining_data

/tmp/ipykernel_35/1034532751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Trigger Words'] = remaining_data['premise'].apply(extract_type4_trigger_words)


,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
10,"If Joseph is a mountain guide, he'll bring his...",Joseph has a climbing harness.,E,possessive,10,type4_possessive_10,his climbing harness
11,"If Theodore is a ski teacher, he'll bring his ...",Theodore has skis.,E,possessive,11,type4_possessive_11,his skis
12,"If David is an actor, he'll memorize his scripts.",David has scripts.,E,possessive,12,type4_possessive_12,his scripts
13,"If Andrew is a home inspector, he'll use his m...",Andrew has a moisture meter.,E,possessive,13,type4_possessive_13,his moisture meter
14,"If James is an athletic trainer, he'll bring h...",James has a first aid kit.,E,possessive,14,type4_possessive_14,his first aid kit
...,...,...,...,...,...,...,...
94,"If Christina is a makeup artist, she'll use he...",Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,her makeup sponge
95,"If Rahim is a clergyman, he'll carry around hi...",Rahim has a holy book.,E,possessive,95,type4_possessive_95,"If Rahim is a clergyman, he'll carry around hi..."
96,"If James is a police officer, he'll leave his ...",James has a gun.,E,possessive,96,type4_possessive_96,his gun
97,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome


In [13]:
type4_all = pd.concat([data_sample,remaining_data])
type4_all

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,his wildlife trap
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,his wetsuit
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,his pastry bag
5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,5,type4_possessive_05,his evidence bags
6,"If Edward is a playwright, he'll bring his pen.",Edward has a pen.,E,possessive,6,type4_possessive_06,his pen
...,...,...,...,...,...,...,...
94,"If Christina is a makeup artist, she'll use he...",Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,her makeup sponge
95,"If Rahim is a clergyman, he'll carry around hi...",Rahim has a holy book.,E,possessive,95,type4_possessive_95,"If Rahim is a clergyman, he'll carry around hi..."
96,"If James is a police officer, he'll leave his ...",James has a gun.,E,possessive,96,type4_possessive_96,his gun
97,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome


In [14]:
type4_all.to_csv('type4_triggers.csv')

In [19]:
def generate_type4_disjunction(original_sentence):

    prompt = (
        f'''Modify the following sentence to be in the form:
        Either [subject] isn't an [occupation], or he [verb] his [tool]. 

        For example: 
        Input: 'If Randolf is a carpenter, he'll use his beading tools for designing.'
        Output: 'Either Randolf isn't a carpenter, or he uses his beading tools for designing.'

        Input: 'If Liam is an aerospace engineer, he'll test the designs in his wind tunnel.'
        Output: 'Either Liam isn't an aerospace engineer, or he tests the designs in his wind tunnel.' 

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [20]:
sentence = generate_type4_disjunction(original_sentence) 
sentence

"Either Bill isn't a content strategist, or he manages everything in his content management application."

In [21]:
data_sample['Disjunction Premise'] = data_sample['premise'].apply(generate_type4_disjunction)
data_sample

/tmp/ipykernel_35/2213440860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Disjunction Premise'] = data_sample['premise'].apply(generate_type4_disjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise
2,"If Steve is a conservation officer, he'll set up his wildlife trap to capture animals for research or relocation.",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,"If Steve is a conservation officer and he sets up his wildlife trap to capture animals for research or relocation, it's okay.","'Either Steve isn't a conservation officer, or he sets up his wildlife trap to capture animals for research or relocation.'"
3,"If Matt is a scuba diver, he'll bring his wetsuit.",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,"If Matt is a scuba diver and he brings his wetsuit, it's okay.","Either Matt isn't a scuba diver, or he brings his wetsuit."
4,"If Noah is a pastry chef, he'll pipe frosting with his pastry bag.",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,"'If Noah is a pastry chef and he pipes frosting with his pastry bag, it's okay.'","'Either Noah isn't a pastry chef, or he pipes frosting with his pastry bag.'"
5,"If Donald is a crime scene investigator, he'll use his evidence bags to collect and secure crucial evidence.",Donald has evidence bags.,E,possessive,5,type4_possessive_05,"If Donald is a crime scene investigator and he uses his evidence bags to collect and secure crucial evidence, it's okay.","'Either Donald isn't a crime scene investigator, or he uses his evidence bags to collect and secure crucial evidence.'"
6,"If Edward is a playwright, he'll bring his pen.",Edward has a pen.,E,possessive,6,type4_possessive_06,"If Edward is a playwright and he brings his pen, it's okay.","'Either Edward isn't a playwright, or he brings his pen.'"
7,"If William is a notary, he'll bring his notary seal.",William has a notary seal.,E,possessive,7,type4_possessive_07,"If William is a notary and he brings his notary seal, it's okay.","Either William isn't a notary, or he brings his notary seal."
8,"If Charles is an urban forester, he'll prune trees with his arborist tools.",Charles has arborist tools.,E,possessive,8,type4_possessive_08,"If Charles is an urban forester and he prunes trees with his arborist tools, it's okay.","'Either Charles isn't an urban forester, or he prunes trees with his arborist tools.'"
9,"If Clyde is a paleontologist, he'll bring his field excavation tools with him.",Clyde has field excavation tools.,E,possessive,9,type4_possessive_09,"If Clyde is a paleontologist and he brings his field excavation tools with him, it's okay.","'Either Clyde isn't a paleontologist, or he brings his field excavation tools with him.'"


In [22]:
remaining_data['Disjunction Premise'] = remaining_data['premise'].apply(generate_type4_disjunction)
remaining_data

/tmp/ipykernel_35/3478991807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Disjunction Premise'] = remaining_data['premise'].apply(generate_type4_disjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise
10,"If Joseph is a mountain guide, he'll bring his climbing harness.",Joseph has a climbing harness.,E,possessive,10,type4_possessive_10,"If Joseph is a mountain guide and he brings his climbing harness, it's okay.","Either Joseph isn't a mountain guide, or he brings his climbing harness."
11,"If Theodore is a ski teacher, he'll bring his skis.",Theodore has skis.,E,possessive,11,type4_possessive_11,"If Theodore is a ski teacher and he brings his skis, it's okay.","Either Theodore isn't a ski teacher, or he brings his skis."
12,"If David is an actor, he'll memorize his scripts.",David has scripts.,E,possessive,12,type4_possessive_12,"If David is an actor and he memorizes his scripts, it's okay.","Either David isn't an actor, or he memorizes his scripts."
13,"If Andrew is a home inspector, he'll use his moisture meter to detect water damage.",Andrew has a moisture meter.,E,possessive,13,type4_possessive_13,"If Andrew is a home inspector and he uses his moisture meter to detect water damage, it's okay.","Either Andrew isn't a home inspector, or he uses his moisture meter to detect water damage."
14,"If James is an athletic trainer, he'll bring his first aid kit.",James has a first aid kit.,E,possessive,14,type4_possessive_14,"If James is an athletic trainer and he brings his first aid kit, it's okay.","Either James isn't an athletic trainer, or he brings his first aid kit."
...,...,...,...,...,...,...,...,...
94,"If Christina is a makeup artist, she'll use her makeup sponge to fix Sara's foundation.",Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,"If Christina is a makeup artist and she uses her makeup sponge to fix Sara's foundation, it's okay.","Either Christina isn't a makeup artist, or she uses her makeup sponge to fix Sara's foundation."
95,"If Rahim is a clergyman, he'll carry around his holy book everywhere.",Rahim has a holy book.,E,possessive,95,type4_possessive_95,"'If Rahim is a clergyman and he carries around his holy book everywhere, it's okay.'","Either Rahim isn't a clergyman, or he carries around his holy book everywhere."
96,"If James is a police officer, he'll leave his gun home.",James has a gun.,E,possessive,96,type4_possessive_96,"If James is a police officer and he leaves his gun at home, it's okay.","Either James isn't a police officer, or he leaves his gun at home."
97,"If Maya is a piano teacher, she'll bring her metronome to school.",Maya has a metronome.,E,possessive,97,type4_possessive_97,"If Maya is a piano teacher and she brings her metronome to school, it's okay.","Either Maya isn't a piano teacher, or she brings her metronome to school."


In [23]:
def generate_type4_attitude(original_sentence):

    prompt = (
        f'''Modify the following sentence to be in the form:
        [subject] believes that his [tool] is [place or adjective]. 

        For example: 
        Input: If Randolf is a carpenter, he'll use his beading tools for designing.
        Output: Randolf believes that his beading tools are upstairs.

        Input: If Liam is an aerospace engineer, he'll test the designs in his wind tunnel.
        Output: Liam believes that his wind tunnel is nearby.

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [24]:
sentence = generate_type4_attitude(original_sentence) 
sentence

'Bill believes that his content management application is efficient.'

In [25]:
data_sample['Attitude Premise'] = data_sample['premise'].apply(generate_type4_attitude)
data_sample

/tmp/ipykernel_35/1026985221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Attitude Premise'] = data_sample['premise'].apply(generate_type4_attitude)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise
2,"If Steve is a conservation officer, he'll set up his wildlife trap to capture animals for research or relocation.",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,"If Steve is a conservation officer and he sets up his wildlife trap to capture animals for research or relocation, it's okay.","'Either Steve isn't a conservation officer, or he sets up his wildlife trap to capture animals for research or relocation.'",Steve believes that his wildlife trap is in the forest.
3,"If Matt is a scuba diver, he'll bring his wetsuit.",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,"If Matt is a scuba diver and he brings his wetsuit, it's okay.","Either Matt isn't a scuba diver, or he brings his wetsuit.",Matt believes that his wetsuit is at home.
4,"If Noah is a pastry chef, he'll pipe frosting with his pastry bag.",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,"'If Noah is a pastry chef and he pipes frosting with his pastry bag, it's okay.'","'Either Noah isn't a pastry chef, or he pipes frosting with his pastry bag.'",Noah believes that his pastry bag is in the pantry.
5,"If Donald is a crime scene investigator, he'll use his evidence bags to collect and secure crucial evidence.",Donald has evidence bags.,E,possessive,5,type4_possessive_05,"If Donald is a crime scene investigator and he uses his evidence bags to collect and secure crucial evidence, it's okay.","'Either Donald isn't a crime scene investigator, or he uses his evidence bags to collect and secure crucial evidence.'",Donald believes that his evidence bags are in the evidence room.
6,"If Edward is a playwright, he'll bring his pen.",Edward has a pen.,E,possessive,6,type4_possessive_06,"If Edward is a playwright and he brings his pen, it's okay.","'Either Edward isn't a playwright, or he brings his pen.'",Edward believes that his pen is essential.
7,"If William is a notary, he'll bring his notary seal.",William has a notary seal.,E,possessive,7,type4_possessive_07,"If William is a notary and he brings his notary seal, it's okay.","Either William isn't a notary, or he brings his notary seal.",William believes that his notary seal is in his briefcase.
8,"If Charles is an urban forester, he'll prune trees with his arborist tools.",Charles has arborist tools.,E,possessive,8,type4_possessive_08,"If Charles is an urban forester and he prunes trees with his arborist tools, it's okay.","'Either Charles isn't an urban forester, or he prunes trees with his arborist tools.'",Charles believes that his arborist tools are in the shed.
9,"If Clyde is a paleontologist, he'll bring his field excavation tools with him.",Clyde has field excavation tools.,E,possessive,9,type4_possessive_09,"If Clyde is a paleontologist and he brings his field excavation tools with him, it's okay.","'Either Clyde isn't a paleontologist, or he brings his field excavation tools with him.'",Clyde believes that his field excavation tools are essential.


In [26]:
remaining_data['Attitude Premise'] = remaining_data['premise'].apply(generate_type4_attitude)
remaining_data

/tmp/ipykernel_35/1366424184.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Attitude Premise'] = remaining_data['premise'].apply(generate_type4_attitude)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise
10,"If Joseph is a mountain guide, he'll bring his climbing harness.",Joseph has a climbing harness.,E,possessive,10,type4_possessive_10,"If Joseph is a mountain guide and he brings his climbing harness, it's okay.","Either Joseph isn't a mountain guide, or he brings his climbing harness.",Joseph believes that his climbing harness is essential.
11,"If Theodore is a ski teacher, he'll bring his skis.",Theodore has skis.,E,possessive,11,type4_possessive_11,"If Theodore is a ski teacher and he brings his skis, it's okay.","Either Theodore isn't a ski teacher, or he brings his skis.",Theodore believes that his skis are in the car.
12,"If David is an actor, he'll memorize his scripts.",David has scripts.,E,possessive,12,type4_possessive_12,"If David is an actor and he memorizes his scripts, it's okay.","Either David isn't an actor, or he memorizes his scripts.",David believes that his scripts are essential.
13,"If Andrew is a home inspector, he'll use his moisture meter to detect water damage.",Andrew has a moisture meter.,E,possessive,13,type4_possessive_13,"If Andrew is a home inspector and he uses his moisture meter to detect water damage, it's okay.","Either Andrew isn't a home inspector, or he uses his moisture meter to detect water damage.",Andrew believes that his moisture meter is essential.
14,"If James is an athletic trainer, he'll bring his first aid kit.",James has a first aid kit.,E,possessive,14,type4_possessive_14,"If James is an athletic trainer and he brings his first aid kit, it's okay.","Either James isn't an athletic trainer, or he brings his first aid kit.",James believes that his first aid kit is in the gym.
...,...,...,...,...,...,...,...,...,...
94,"If Christina is a makeup artist, she'll use her makeup sponge to fix Sara's foundation.",Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,"If Christina is a makeup artist and she uses her makeup sponge to fix Sara's foundation, it's okay.","Either Christina isn't a makeup artist, or she uses her makeup sponge to fix Sara's foundation.",Christina believes that her makeup sponge is in the makeup kit.
95,"If Rahim is a clergyman, he'll carry around his holy book everywhere.",Rahim has a holy book.,E,possessive,95,type4_possessive_95,"'If Rahim is a clergyman and he carries around his holy book everywhere, it's okay.'","Either Rahim isn't a clergyman, or he carries around his holy book everywhere.",Rahim believes that his holy book is sacred.
96,"If James is a police officer, he'll leave his gun home.",James has a gun.,E,possessive,96,type4_possessive_96,"If James is a police officer and he leaves his gun at home, it's okay.","Either James isn't a police officer, or he leaves his gun at home.",James believes that his gun is secure.
97,"If Maya is a piano teacher, she'll bring her metronome to school.",Maya has a metronome.,E,possessive,97,type4_possessive_97,"If Maya is a piano teacher and she brings her metronome to school, it's okay.","Either Maya isn't a piano teacher, or she brings her metronome to school.",Maya believes that her metronome is in the classroom.


In [27]:
type4_all = pd.concat([data_sample,remaining_data])
type4_all

,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise
2,"If Steve is a conservation officer, he'll set up his wildlife trap to capture animals for research or relocation.",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,"If Steve is a conservation officer and he sets up his wildlife trap to capture animals for research or relocation, it's okay.","'Either Steve isn't a conservation officer, or he sets up his wildlife trap to capture animals for research or relocation.'",Steve believes that his wildlife trap is in the forest.
3,"If Matt is a scuba diver, he'll bring his wetsuit.",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,"If Matt is a scuba diver and he brings his wetsuit, it's okay.","Either Matt isn't a scuba diver, or he brings his wetsuit.",Matt believes that his wetsuit is at home.
4,"If Noah is a pastry chef, he'll pipe frosting with his pastry bag.",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,"'If Noah is a pastry chef and he pipes frosting with his pastry bag, it's okay.'","'Either Noah isn't a pastry chef, or he pipes frosting with his pastry bag.'",Noah believes that his pastry bag is in the pantry.
5,"If Donald is a crime scene investigator, he'll use his evidence bags to collect and secure crucial evidence.",Donald has evidence bags.,E,possessive,5,type4_possessive_05,"If Donald is a crime scene investigator and he uses his evidence bags to collect and secure crucial evidence, it's okay.","'Either Donald isn't a crime scene investigator, or he uses his evidence bags to collect and secure crucial evidence.'",Donald believes that his evidence bags are in the evidence room.
6,"If Edward is a playwright, he'll bring his pen.",Edward has a pen.,E,possessive,6,type4_possessive_06,"If Edward is a playwright and he brings his pen, it's okay.","'Either Edward isn't a playwright, or he brings his pen.'",Edward believes that his pen is essential.
...,...,...,...,...,...,...,...,...,...
94,"If Christina is a makeup artist, she'll use her makeup sponge to fix Sara's foundation.",Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,"If Christina is a makeup artist and she uses her makeup sponge to fix Sara's foundation, it's okay.","Either Christina isn't a makeup artist, or she uses her makeup sponge to fix Sara's foundation.",Christina believes that her makeup sponge is in the makeup kit.
95,"If Rahim is a clergyman, he'll carry around his holy book everywhere.",Rahim has a holy book.,E,possessive,95,type4_possessive_95,"'If Rahim is a clergyman and he carries around his holy book everywhere, it's okay.'","Either Rahim isn't a clergyman, or he carries around his holy book everywhere.",Rahim believes that his holy book is sacred.
96,"If James is a police officer, he'll leave his gun home.",James has a gun.,E,possessive,96,type4_possessive_96,"If James is a police officer and he leaves his gun at home, it's okay.","Either James isn't a police officer, or he leaves his gun at home.",James believes that his gun is secure.
97,"If Maya is a piano teacher, she'll bring her metronome to school.",Maya has a metronome.,E,possessive,97,type4_possessive_97,"If Maya is a piano teacher and she brings her metronome to school, it's okay.","Either Maya isn't a piano teacher, or she brings her metronome to school.",Maya believes that her metronome is in the classroom.


In [28]:
type4_all.to_csv('part4a_type4.csv')

## Type 5 Possessive

In [15]:
type5p_data = pd.read_excel('/kaggle/input/confer-extension/Raw_Dataset/Type5_possessive1_filtered.xlsx')
type5p_data

,premise,hypothesis,gold_label,trigger,SetID,UID
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,0,type5_possessive_1_00
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,1,type5_possessive_1_01
2,"If Scarlett finishes her work early, her husba...",Scarlett has a husband.,E,possessive,2,type5_possessive_1_02
3,"If Chloe finishes his work early, his colleagu...",Chloe has colleagues.,E,possessive,3,type5_possessive_1_03
4,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,possessive,4,type5_possessive_1_04
...,...,...,...,...,...,...
395,"If Dorothy car breaks down, her brother will c...",Dorothy has a brother.,E,possessive,395,type5_possessive_1_395
396,"If Susan finishes his work early, his colleagu...",Susan has a colleague.,E,possessive,396,type5_possessive_1_396
397,"If Margaret finishes work early, his wife will...",Margaret has a wife.,E,possessive,397,type5_possessive_1_397
398,"If Betty misses the bus, her brother will driv...",Betty has a brother.,E,possessive,398,type5_possessive_1_398


In [4]:
def generate_type5_conjunction(original_sentence):

    prompt = (
        f'''Modify the following sentence to be in the form:
        If [antecedent] and [consequent], it's okay. 

        For example: 
        Input: 'If Sarah attends the conference, her friend will meet her at the hotel.'
        Output: 'If Sarah attends the conference and her friend meets her at the hotel, it's okay.'

        Input: 'If Emily passes her driving test, her best friend will take her out for a celebratory dinner.'
        Output: 'If Emily passes her driving test and her best friend takes her out for a celebratory dinner, it's okay.' 

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [6]:
original_sentence = type5p_data['premise'].iloc[0]
original_sentence

'If Jessica attends the conference, her colleague will meet her at the hotel.'

In [7]:
sentence = generate_type5_conjunction(original_sentence) 
sentence

"If Jessica attends the conference and her colleague meets her at the hotel, it's okay."

In [21]:
data_sample = type5p_data[2:10]
data_sample

,premise,hypothesis,gold_label,trigger,SetID,UID
2,"If Scarlett finishes her work early, her husba...",Scarlett has a husband.,E,possessive,2,type5_possessive_1_02
3,"If Chloe finishes his work early, his colleagu...",Chloe has colleagues.,E,possessive,3,type5_possessive_1_03
4,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,possessive,4,type5_possessive_1_04
5,"If Mia's car breaks down, her coworker will gi...",Mia has a coworker.,E,possessive,5,type5_possessive_1_05
6,"If Ryan passes his driving test, his parents w...",Ryan has parents.,E,possessive,6,type5_possessive_1_06
7,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,possessive,7,type5_possessive_1_07
8,"If Lisa finishes his meeting early, his collea...",Lisa has a colleague.,E,possessive,8,type5_possessive_1_08
9,"If Jennifer finishes her presentation early, h...",Jennifer has colleagues.,E,possessive,9,type5_possessive_1_09


In [10]:
data_sample['Conjunction Premise'] = data_sample['premise'].apply(generate_type5_conjunction)
data_sample

/tmp/ipykernel_35/1355896315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Conjunction Premise'] = data_sample['premise'].apply(generate_type5_conjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise
2,"If Scarlett finishes her work early, her husba...",Scarlett has a husband.,E,possessive,2,type5_possessive_1_02,If Scarlett finishes her work early and her hu...
3,"If Chloe finishes his work early, his colleagu...",Chloe has colleagues.,E,possessive,3,type5_possessive_1_03,If Chloe finishes his work early and his colle...
4,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,possessive,4,type5_possessive_1_04,'If Charlotte passes her driving test and her ...
5,"If Mia's car breaks down, her coworker will gi...",Mia has a coworker.,E,possessive,5,type5_possessive_1_05,'If Mia's car breaks down and her coworker giv...
6,"If Ryan passes his driving test, his parents w...",Ryan has parents.,E,possessive,6,type5_possessive_1_06,If Ryan passes his driving test and his parent...
7,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,possessive,7,type5_possessive_1_07,If Lily takes the train to the city and her fr...
8,"If Lisa finishes his meeting early, his collea...",Lisa has a colleague.,E,possessive,8,type5_possessive_1_08,If Lisa finishes his meeting early and his col...
9,"If Jennifer finishes her presentation early, h...",Jennifer has colleagues.,E,possessive,9,type5_possessive_1_09,If Jennifer finishes her presentation early an...


In [11]:
pd.set_option('display.max_colwidth', None)
data_sample

,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise
2,"If Scarlett finishes her work early, her husband will take her out for dinner.",Scarlett has a husband.,E,possessive,2,type5_possessive_1_02,"If Scarlett finishes her work early and her husband takes her out for dinner, it's okay."
3,"If Chloe finishes his work early, his colleagues will take him out for lunch.",Chloe has colleagues.,E,possessive,3,type5_possessive_1_03,"If Chloe finishes his work early and his colleagues take him out for lunch, it's okay."
4,"If Charlotte passes her driving test, her parents will buy her a car.",Charlotte has parents.,E,possessive,4,type5_possessive_1_04,"'If Charlotte passes her driving test and her parents buy her a car, it's okay.'"
5,"If Mia's car breaks down, her coworker will give her a ride to the office.",Mia has a coworker.,E,possessive,5,type5_possessive_1_05,"'If Mia's car breaks down and her coworker gives her a ride to the office, it's okay.'"
6,"If Ryan passes his driving test, his parents will be very happy.",Ryan has parents.,E,possessive,6,type5_possessive_1_06,"If Ryan passes his driving test and his parents are very happy, it's okay."
7,"If Lily takes the train to the city, her friend will meet her at the station.",Lily has a friend.,E,possessive,7,type5_possessive_1_07,"If Lily takes the train to the city and her friend meets her at the station, it's okay."
8,"If Lisa finishes his meeting early, his colleague will drive him to the conference.",Lisa has a colleague.,E,possessive,8,type5_possessive_1_08,"If Lisa finishes his meeting early and his colleague drives him to the conference, it's okay."
9,"If Jennifer finishes her presentation early, her colleagues will take her out for lunch.",Jennifer has colleagues.,E,possessive,9,type5_possessive_1_09,"If Jennifer finishes her presentation early and her colleagues take her out for lunch, it's okay."


In [22]:
remaining_data = type5p_data[10:]
remaining_data

,premise,hypothesis,gold_label,trigger,SetID,UID
10,"If Sarah has a doctor's appointment, her husba...",Sarah has a husband.,E,possessive,10,type5_possessive_1_10
11,"If Evelyn lands in New York, her best friend w...",Evelyn has a best friend.,E,possessive,11,type5_possessive_1_11
12,"If Daniel arrives on time, his best friend wil...",Daniel has a best friend.,E,possessive,12,type5_possessive_1_12
13,"If Isabella finishes her work early, her colle...",Isabella has a colleague.,E,possessive,13,type5_possessive_1_13
14,"If Elijah attends the conference, his colleagu...",Elijah has a colleague.,E,possessive,14,type5_possessive_1_14
...,...,...,...,...,...,...
395,"If Dorothy car breaks down, her brother will c...",Dorothy has a brother.,E,possessive,395,type5_possessive_1_395
396,"If Susan finishes his work early, his colleagu...",Susan has a colleague.,E,possessive,396,type5_possessive_1_396
397,"If Margaret finishes work early, his wife will...",Margaret has a wife.,E,possessive,397,type5_possessive_1_397
398,"If Betty misses the bus, her brother will driv...",Betty has a brother.,E,possessive,398,type5_possessive_1_398


In [14]:
remaining_data['Conjunction Premise'] = remaining_data['premise'].apply(generate_type5_conjunction)
remaining_data

/tmp/ipykernel_35/3843941919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Conjunction Premise'] = remaining_data['premise'].apply(generate_type5_conjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise
10,"If Sarah has a doctor's appointment, her husband will drive her to the clinic.",Sarah has a husband.,E,possessive,10,type5_possessive_1_10,"If Sarah has a doctor's appointment and her husband drives her to the clinic, it's okay."
11,"If Evelyn lands in New York, her best friend will greet her at the airport.",Evelyn has a best friend.,E,possessive,11,type5_possessive_1_11,"If Evelyn lands in New York and her best friend greets her at the airport, it's okay."
12,"If Daniel arrives on time, his best friend will be waiting for him at the train station.",Daniel has a best friend.,E,possessive,12,type5_possessive_1_12,"If Daniel arrives on time and his best friend is waiting for him at the train station, it's okay."
13,"If Isabella finishes her work early, her colleague will treat her to lunch.",Isabella has a colleague.,E,possessive,13,type5_possessive_1_13,"If Isabella finishes her work early and her colleague treats her to lunch, it's okay."
14,"If Elijah attends the conference, his colleague will meet him at the hotel.",Elijah has a colleague.,E,possessive,14,type5_possessive_1_14,"'If Elijah attends the conference and his colleague meets him at the hotel, it's okay.'"
...,...,...,...,...,...,...,...
395,"If Dorothy car breaks down, her brother will come to give her a ride home.",Dorothy has a brother.,E,possessive,395,type5_possessive_1_395,"If Dorothy's car breaks down and her brother comes to give her a ride home, it's okay."
396,"If Susan finishes his work early, his colleague will drive him home.",Susan has a colleague.,E,possessive,396,type5_possessive_1_396,"If Susan finishes his work early and his colleague drives him home, it's okay."
397,"If Margaret finishes work early, his wife will cook dinner for him.",Margaret has a wife.,E,possessive,397,type5_possessive_1_397,"If Margaret finishes work early and his wife cooks dinner for him, it's okay."
398,"If Betty misses the bus, her brother will drive her to school.",Betty has a brother.,E,possessive,398,type5_possessive_1_398,"If Betty misses the bus and her brother drives her to school, it's okay."


In [19]:
def generate_type5_disjunction(original_sentence):

    prompt = (
        f'''Modify the following sentence to be in the form:
        Either [subject] doesn't [action], or he and his [person] [action]. 

        For example: 
        Input: If Sarah attends the conference, her friend will meet her at the hotel.
        Output: Either Sarah doesn't attend the conference, or she and her friend do.

        Input: If Emily passes her driving test, her best friend will take her out for a celebratory dinner.
        Output: Either Emily doesn't pass her driving test, or she and her best friend go out.

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [17]:
sentence = generate_type5_disjunction(original_sentence) 
sentence

"Either Jessica doesn't attend the conference, or she and her colleague do."

In [20]:
data_sample['Disjunction Premise'] = data_sample['premise'].apply(generate_type5_disjunction)
data_sample

/tmp/ipykernel_35/3999524361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Disjunction Premise'] = data_sample['premise'].apply(generate_type5_disjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise
2,"If Scarlett finishes her work early, her husband will take her out for dinner.",Scarlett has a husband.,E,possessive,2,type5_possessive_1_02,"If Scarlett finishes her work early and her husband takes her out for dinner, it's okay.","Either Scarlett doesn't finish her work early, or she and her husband go out for dinner."
3,"If Chloe finishes his work early, his colleagues will take him out for lunch.",Chloe has colleagues.,E,possessive,3,type5_possessive_1_03,"If Chloe finishes his work early and his colleagues take him out for lunch, it's okay.","Either Chloe doesn't finish his work early, or he and his colleagues go out for lunch."
4,"If Charlotte passes her driving test, her parents will buy her a car.",Charlotte has parents.,E,possessive,4,type5_possessive_1_04,"'If Charlotte passes her driving test and her parents buy her a car, it's okay.'","Either Charlotte doesn't pass her driving test, or she and her parents buy a car."
5,"If Mia's car breaks down, her coworker will give her a ride to the office.",Mia has a coworker.,E,possessive,5,type5_possessive_1_05,"'If Mia's car breaks down and her coworker gives her a ride to the office, it's okay.'","Either Mia's car doesn't break down, or she and her coworker ride to the office together."
6,"If Ryan passes his driving test, his parents will be very happy.",Ryan has parents.,E,possessive,6,type5_possessive_1_06,"If Ryan passes his driving test and his parents are very happy, it's okay.","Either Ryan doesn't pass his driving test, or he and his parents will be very happy."
7,"If Lily takes the train to the city, her friend will meet her at the station.",Lily has a friend.,E,possessive,7,type5_possessive_1_07,"If Lily takes the train to the city and her friend meets her at the station, it's okay.","Either Lily doesn't take the train to the city, or she and her friend meet at the station."
8,"If Lisa finishes his meeting early, his colleague will drive him to the conference.",Lisa has a colleague.,E,possessive,8,type5_possessive_1_08,"If Lisa finishes his meeting early and his colleague drives him to the conference, it's okay.","Either Lisa doesn't finish his meeting early, or he and his colleague drive to the conference."
9,"If Jennifer finishes her presentation early, her colleagues will take her out for lunch.",Jennifer has colleagues.,E,possessive,9,type5_possessive_1_09,"If Jennifer finishes her presentation early and her colleagues take her out for lunch, it's okay.","Either Jennifer doesn't finish her presentation early, or she and her colleagues go out for lunch."


In [21]:
remaining_data['Disjunction Premise'] = remaining_data['premise'].apply(generate_type5_disjunction)
remaining_data

/tmp/ipykernel_35/1787260321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Disjunction Premise'] = remaining_data['premise'].apply(generate_type5_disjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise
10,"If Sarah has a doctor's appointment, her husband will drive her to the clinic.",Sarah has a husband.,E,possessive,10,type5_possessive_1_10,"If Sarah has a doctor's appointment and her husband drives her to the clinic, it's okay.","Either Sarah doesn't have a doctor's appointment, or she and her husband go to the clinic together."
11,"If Evelyn lands in New York, her best friend will greet her at the airport.",Evelyn has a best friend.,E,possessive,11,type5_possessive_1_11,"If Evelyn lands in New York and her best friend greets her at the airport, it's okay.","Either Evelyn doesn't land in New York, or she and her best friend don't meet at the airport."
12,"If Daniel arrives on time, his best friend will be waiting for him at the train station.",Daniel has a best friend.,E,possessive,12,type5_possessive_1_12,"If Daniel arrives on time and his best friend is waiting for him at the train station, it's okay.","Either Daniel doesn't arrive on time, or he and his best friend do."
13,"If Isabella finishes her work early, her colleague will treat her to lunch.",Isabella has a colleague.,E,possessive,13,type5_possessive_1_13,"If Isabella finishes her work early and her colleague treats her to lunch, it's okay.","Either Isabella doesn't finish her work early, or she and her colleague go out for lunch."
14,"If Elijah attends the conference, his colleague will meet him at the hotel.",Elijah has a colleague.,E,possessive,14,type5_possessive_1_14,"'If Elijah attends the conference and his colleague meets him at the hotel, it's okay.'","Either Elijah doesn't attend the conference, or he and his colleague meet at the hotel."
...,...,...,...,...,...,...,...,...
395,"If Dorothy car breaks down, her brother will come to give her a ride home.",Dorothy has a brother.,E,possessive,395,type5_possessive_1_395,"If Dorothy's car breaks down and her brother comes to give her a ride home, it's okay.","Either Dorothy's car doesn't break down, or she and her brother go out."
396,"If Susan finishes his work early, his colleague will drive him home.",Susan has a colleague.,E,possessive,396,type5_possessive_1_396,"If Susan finishes his work early and his colleague drives him home, it's okay.","Either Susan doesn't finish his work early, or he and his colleague drive home together."
397,"If Margaret finishes work early, his wife will cook dinner for him.",Margaret has a wife.,E,possessive,397,type5_possessive_1_397,"If Margaret finishes work early and his wife cooks dinner for him, it's okay.","Either Margaret doesn't finish work early, or he and his wife cook dinner together."
398,"If Betty misses the bus, her brother will drive her to school.",Betty has a brother.,E,possessive,398,type5_possessive_1_398,"If Betty misses the bus and her brother drives her to school, it's okay.","Either Betty doesn't miss the bus, or she and her brother drive to school."


In [23]:
def generate_type5_attitude(original_sentence):

    prompt = (
        f'''Modify the following sentence to be in the form:
        [subject] believes that his [person] is [place or adjective]. 

        For example: 
        Input: If Sarah attends the conference, her friend will meet her at the hotel.
        Output: Sarah believes that her friend is upstairs.

        Input: If Emily passes her driving test, her best friend will take her out for a celebratory dinner.
        Output: Emily believes her best friend is nearby.

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [24]:
sentence = generate_type5_attitude(original_sentence) 
sentence

'Jessica believes that her colleague is waiting.'

In [25]:
data_sample['Attitude Premise'] = data_sample['premise'].apply(generate_type5_attitude)
data_sample

/tmp/ipykernel_35/490184230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Attitude Premise'] = data_sample['premise'].apply(generate_type5_attitude)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise
2,"If Scarlett finishes her work early, her husband will take her out for dinner.",Scarlett has a husband.,E,possessive,2,type5_possessive_1_02,"If Scarlett finishes her work early and her husband takes her out for dinner, it's okay.","Either Scarlett doesn't finish her work early, or she and her husband go out for dinner.",Scarlett believes that her husband is waiting.
3,"If Chloe finishes his work early, his colleagues will take him out for lunch.",Chloe has colleagues.,E,possessive,3,type5_possessive_1_03,"If Chloe finishes his work early and his colleagues take him out for lunch, it's okay.","Either Chloe doesn't finish his work early, or he and his colleagues go out for lunch.",Chloe believes that his colleagues are at the restaurant.
4,"If Charlotte passes her driving test, her parents will buy her a car.",Charlotte has parents.,E,possessive,4,type5_possessive_1_04,"'If Charlotte passes her driving test and her parents buy her a car, it's okay.'","Either Charlotte doesn't pass her driving test, or she and her parents buy a car.",Charlotte believes that her parents are generous.
5,"If Mia's car breaks down, her coworker will give her a ride to the office.",Mia has a coworker.,E,possessive,5,type5_possessive_1_05,"'If Mia's car breaks down and her coworker gives her a ride to the office, it's okay.'","Either Mia's car doesn't break down, or she and her coworker ride to the office together.",Mia believes that her coworker is reliable.
6,"If Ryan passes his driving test, his parents will be very happy.",Ryan has parents.,E,possessive,6,type5_possessive_1_06,"If Ryan passes his driving test and his parents are very happy, it's okay.","Either Ryan doesn't pass his driving test, or he and his parents will be very happy.",Ryan believes that his parents are joyful.
7,"If Lily takes the train to the city, her friend will meet her at the station.",Lily has a friend.,E,possessive,7,type5_possessive_1_07,"If Lily takes the train to the city and her friend meets her at the station, it's okay.","Either Lily doesn't take the train to the city, or she and her friend meet at the station.",Lily believes her friend is waiting at the station.
8,"If Lisa finishes his meeting early, his colleague will drive him to the conference.",Lisa has a colleague.,E,possessive,8,type5_possessive_1_08,"If Lisa finishes his meeting early and his colleague drives him to the conference, it's okay.","Either Lisa doesn't finish his meeting early, or he and his colleague drive to the conference.",Lisa believes that his colleague is waiting outside.
9,"If Jennifer finishes her presentation early, her colleagues will take her out for lunch.",Jennifer has colleagues.,E,possessive,9,type5_possessive_1_09,"If Jennifer finishes her presentation early and her colleagues take her out for lunch, it's okay.","Either Jennifer doesn't finish her presentation early, or she and her colleagues go out for lunch.",Jennifer believes that her colleagues are waiting.


In [26]:
remaining_data['Attitude Premise'] = remaining_data['premise'].apply(generate_type5_attitude)
remaining_data

/tmp/ipykernel_35/1927655460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Attitude Premise'] = remaining_data['premise'].apply(generate_type5_attitude)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise
10,"If Sarah has a doctor's appointment, her husband will drive her to the clinic.",Sarah has a husband.,E,possessive,10,type5_possessive_1_10,"If Sarah has a doctor's appointment and her husband drives her to the clinic, it's okay.","Either Sarah doesn't have a doctor's appointment, or she and her husband go to the clinic together.",Sarah believes that her husband is waiting.
11,"If Evelyn lands in New York, her best friend will greet her at the airport.",Evelyn has a best friend.,E,possessive,11,type5_possessive_1_11,"If Evelyn lands in New York and her best friend greets her at the airport, it's okay.","Either Evelyn doesn't land in New York, or she and her best friend don't meet at the airport.",Evelyn believes that her best friend is waiting at the airport.
12,"If Daniel arrives on time, his best friend will be waiting for him at the train station.",Daniel has a best friend.,E,possessive,12,type5_possessive_1_12,"If Daniel arrives on time and his best friend is waiting for him at the train station, it's okay.","Either Daniel doesn't arrive on time, or he and his best friend do.",Daniel believes that his best friend is at the train station.
13,"If Isabella finishes her work early, her colleague will treat her to lunch.",Isabella has a colleague.,E,possessive,13,type5_possessive_1_13,"If Isabella finishes her work early and her colleague treats her to lunch, it's okay.","Either Isabella doesn't finish her work early, or she and her colleague go out for lunch.",Isabella believes that her colleague is generous.
14,"If Elijah attends the conference, his colleague will meet him at the hotel.",Elijah has a colleague.,E,possessive,14,type5_possessive_1_14,"'If Elijah attends the conference and his colleague meets him at the hotel, it's okay.'","Either Elijah doesn't attend the conference, or he and his colleague meet at the hotel.",Elijah believes that his colleague is waiting at the hotel.
...,...,...,...,...,...,...,...,...,...
395,"If Dorothy car breaks down, her brother will come to give her a ride home.",Dorothy has a brother.,E,possessive,395,type5_possessive_1_395,"If Dorothy's car breaks down and her brother comes to give her a ride home, it's okay.","Either Dorothy's car doesn't break down, or she and her brother go out.",Dorothy believes that her brother is on the way.
396,"If Susan finishes his work early, his colleague will drive him home.",Susan has a colleague.,E,possessive,396,type5_possessive_1_396,"If Susan finishes his work early and his colleague drives him home, it's okay.","Either Susan doesn't finish his work early, or he and his colleague drive home together.",Susan believes that his colleague is waiting.
397,"If Margaret finishes work early, his wife will cook dinner for him.",Margaret has a wife.,E,possessive,397,type5_possessive_1_397,"If Margaret finishes work early and his wife cooks dinner for him, it's okay.","Either Margaret doesn't finish work early, or he and his wife cook dinner together.",Margaret believes that his wife is at home.
398,"If Betty misses the bus, her brother will drive her to school.",Betty has a brother.,E,possessive,398,type5_possessive_1_398,"If Betty misses the bus and her brother drives her to school, it's okay.","Either Betty doesn't miss the bus, or she and her brother drive to school.",Betty believes that her brother is waiting outside.


In [27]:
type5_all = pd.concat([data_sample,remaining_data])
type5_all

,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise
2,"If Scarlett finishes her work early, her husband will take her out for dinner.",Scarlett has a husband.,E,possessive,2,type5_possessive_1_02,"If Scarlett finishes her work early and her husband takes her out for dinner, it's okay.","Either Scarlett doesn't finish her work early, or she and her husband go out for dinner.",Scarlett believes that her husband is waiting.
3,"If Chloe finishes his work early, his colleagues will take him out for lunch.",Chloe has colleagues.,E,possessive,3,type5_possessive_1_03,"If Chloe finishes his work early and his colleagues take him out for lunch, it's okay.","Either Chloe doesn't finish his work early, or he and his colleagues go out for lunch.",Chloe believes that his colleagues are at the restaurant.
4,"If Charlotte passes her driving test, her parents will buy her a car.",Charlotte has parents.,E,possessive,4,type5_possessive_1_04,"'If Charlotte passes her driving test and her parents buy her a car, it's okay.'","Either Charlotte doesn't pass her driving test, or she and her parents buy a car.",Charlotte believes that her parents are generous.
5,"If Mia's car breaks down, her coworker will give her a ride to the office.",Mia has a coworker.,E,possessive,5,type5_possessive_1_05,"'If Mia's car breaks down and her coworker gives her a ride to the office, it's okay.'","Either Mia's car doesn't break down, or she and her coworker ride to the office together.",Mia believes that her coworker is reliable.
6,"If Ryan passes his driving test, his parents will be very happy.",Ryan has parents.,E,possessive,6,type5_possessive_1_06,"If Ryan passes his driving test and his parents are very happy, it's okay.","Either Ryan doesn't pass his driving test, or he and his parents will be very happy.",Ryan believes that his parents are joyful.
...,...,...,...,...,...,...,...,...,...
395,"If Dorothy car breaks down, her brother will come to give her a ride home.",Dorothy has a brother.,E,possessive,395,type5_possessive_1_395,"If Dorothy's car breaks down and her brother comes to give her a ride home, it's okay.","Either Dorothy's car doesn't break down, or she and her brother go out.",Dorothy believes that her brother is on the way.
396,"If Susan finishes his work early, his colleague will drive him home.",Susan has a colleague.,E,possessive,396,type5_possessive_1_396,"If Susan finishes his work early and his colleague drives him home, it's okay.","Either Susan doesn't finish his work early, or he and his colleague drive home together.",Susan believes that his colleague is waiting.
397,"If Margaret finishes work early, his wife will cook dinner for him.",Margaret has a wife.,E,possessive,397,type5_possessive_1_397,"If Margaret finishes work early and his wife cooks dinner for him, it's okay.","Either Margaret doesn't finish work early, or he and his wife cook dinner together.",Margaret believes that his wife is at home.
398,"If Betty misses the bus, her brother will drive her to school.",Betty has a brother.,E,possessive,398,type5_possessive_1_398,"If Betty misses the bus and her brother drives her to school, it's okay.","Either Betty doesn't miss the bus, or she and her brother drive to school.",Betty believes that her brother is waiting outside.


In [28]:
type5_all.to_csv('part4a_type5p.csv')

In [18]:
def extract_type5_trigger_words(original_sentence):

    prompt = (
        f'''Extract the possessive phrase from the following sentence.

        For example: 
        Input: If Sarah attends the conference, her friend will meet her at the hotel.
        Output: her friend

        Input: If Jessica attends the conference, her colleague will meet her at the hotel.
        Output: her colleague

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [16]:
original_sentence = type5p_data['premise'].iloc[1]
original_sentence

'If Emily passes her driving test, her best friend will take her out for a celebratory dinner.'

In [19]:
extract_type5_trigger_words(original_sentence) 

'her best friend'

In [23]:
data_sample['Trigger Words'] = data_sample['premise'].apply(extract_type5_trigger_words)
data_sample

/tmp/ipykernel_35/323511182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Trigger Words'] = data_sample['premise'].apply(extract_type5_trigger_words)


,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
2,"If Scarlett finishes her work early, her husba...",Scarlett has a husband.,E,possessive,2,type5_possessive_1_02,her husband
3,"If Chloe finishes his work early, his colleagu...",Chloe has colleagues.,E,possessive,3,type5_possessive_1_03,his colleagues
4,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,possessive,4,type5_possessive_1_04,her parents
5,"If Mia's car breaks down, her coworker will gi...",Mia has a coworker.,E,possessive,5,type5_possessive_1_05,Mia's car
6,"If Ryan passes his driving test, his parents w...",Ryan has parents.,E,possessive,6,type5_possessive_1_06,his parents
7,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,possessive,7,type5_possessive_1_07,her friend
8,"If Lisa finishes his meeting early, his collea...",Lisa has a colleague.,E,possessive,8,type5_possessive_1_08,his colleague
9,"If Jennifer finishes her presentation early, h...",Jennifer has colleagues.,E,possessive,9,type5_possessive_1_09,her colleagues


In [24]:
remaining_data['Trigger Words'] = remaining_data['premise'].apply(extract_type5_trigger_words)
remaining_data

/tmp/ipykernel_35/4027204886.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Trigger Words'] = remaining_data['premise'].apply(extract_type5_trigger_words)


,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
10,"If Sarah has a doctor's appointment, her husba...",Sarah has a husband.,E,possessive,10,type5_possessive_1_10,her husband
11,"If Evelyn lands in New York, her best friend w...",Evelyn has a best friend.,E,possessive,11,type5_possessive_1_11,her best friend
12,"If Daniel arrives on time, his best friend wil...",Daniel has a best friend.,E,possessive,12,type5_possessive_1_12,his best friend
13,"If Isabella finishes her work early, her colle...",Isabella has a colleague.,E,possessive,13,type5_possessive_1_13,her colleague
14,"If Elijah attends the conference, his colleagu...",Elijah has a colleague.,E,possessive,14,type5_possessive_1_14,his colleague
...,...,...,...,...,...,...,...
395,"If Dorothy car breaks down, her brother will c...",Dorothy has a brother.,E,possessive,395,type5_possessive_1_395,her brother
396,"If Susan finishes his work early, his colleagu...",Susan has a colleague.,E,possessive,396,type5_possessive_1_396,his colleague
397,"If Margaret finishes work early, his wife will...",Margaret has a wife.,E,possessive,397,type5_possessive_1_397,his wife
398,"If Betty misses the bus, her brother will driv...",Betty has a brother.,E,possessive,398,type5_possessive_1_398,her brother


In [26]:
type5_all = pd.concat([data_sample,remaining_data])
type5_all

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
2,"If Scarlett finishes her work early, her husba...",Scarlett has a husband.,E,possessive,2,type5_possessive_1_02,her husband
3,"If Chloe finishes his work early, his colleagu...",Chloe has colleagues.,E,possessive,3,type5_possessive_1_03,his colleagues
4,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,possessive,4,type5_possessive_1_04,her parents
5,"If Mia's car breaks down, her coworker will gi...",Mia has a coworker.,E,possessive,5,type5_possessive_1_05,Mia's car
6,"If Ryan passes his driving test, his parents w...",Ryan has parents.,E,possessive,6,type5_possessive_1_06,his parents
...,...,...,...,...,...,...,...
395,"If Dorothy car breaks down, her brother will c...",Dorothy has a brother.,E,possessive,395,type5_possessive_1_395,her brother
396,"If Susan finishes his work early, his colleagu...",Susan has a colleague.,E,possessive,396,type5_possessive_1_396,his colleague
397,"If Margaret finishes work early, his wife will...",Margaret has a wife.,E,possessive,397,type5_possessive_1_397,his wife
398,"If Betty misses the bus, her brother will driv...",Betty has a brother.,E,possessive,398,type5_possessive_1_398,her brother


In [27]:
type5_all.to_csv('type5p_triggers.csv')

## Type 5 Again

In [29]:
type5a_data = pd.read_excel('/kaggle/input/confer-extension/Raw_Dataset/Type5_possessive2_filtered.xlsx')
type5a_data

,premise,hypothesis,gold_label,trigger,SetID,UID
0,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00
1,"If Emily passes her driving test, she'll never...",Emily has read a book by Dan Brown before.,E,possessive,1,type5_again_2_01
2,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,possessive,2,type5_again_2_02
3,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03
4,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04
...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395
396,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396
397,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397
398,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398


In [8]:
def generate_type5a_conjunction(original_sentence):

    prompt = (
        f'''Modify the following sentence to be in the form:
        If [antecedent] and [consequent], it's okay.  

        For example: 
        Input: 'If Sarah attends the conference, she'll never watch Star Wars again.'
        Output: 'If Sarah attends the conference and she never watches Star Wars again, it's okay.'

        Input: 'If Emily passes her driving test, she'll never read a book by Dan Brown again.'
        Output: 'If Emily passes her driving test and she never reads a book by Dan Brown again, it's okay.' 

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [25]:
def extract_type5a_trigger_words(original_sentence):

    prompt = (
        f'''Extract the phrase "[instance] again" from the provided sentence

        For example: 
        Input: 'If Sarah attends the conference, she'll never watch Star Wars again.'
        Output: Star Wars again

        Input: 'If Emily passes her driving test, she'll never read a book by Dan Brown again.'
        Output: Dan Brown again

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [30]:
original_sentence = type5a_data['premise'].iloc[0]
original_sentence

"If Jessica attends the conference, she'll never watch a movie by Nolan again."

In [9]:
sentence = generate_type5a_conjunction(original_sentence) 
sentence

"If Jessica attends the conference and she never watches a movie by Nolan again, it's okay."

In [31]:
data_sample = type5a_data[2:10]
data_sample 

,premise,hypothesis,gold_label,trigger,SetID,UID
2,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,possessive,2,type5_again_2_02
3,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03
4,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04
5,"If Mia's car breaks down, she'll never write a...",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05
6,"If Ryan passes his driving test, he'll never r...",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06
7,"If Lily takes the train to the city, she'll ne...",Lily has painted a portrait before.,E,possessive,7,type5_again_2_07
8,"If Lisa finishes her meeting early, she'll nev...",Lisa has driven a sports car before.,E,possessive,8,type5_again_2_08
9,"If Jennifer finishes her presentation early, s...",Jennifer has played a musical instrument before.,E,possessive,9,type5_again_2_09


In [11]:
data_sample['Conjunction Premise'] = data_sample['premise'].apply(generate_type5a_conjunction)
data_sample

/tmp/ipykernel_35/773504816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Conjunction Premise'] = data_sample['premise'].apply(generate_type5a_conjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise
2,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,possessive,2,type5_again_2_02,If Scarlett finishes her work early and she ne...
3,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,If Chloe finishes her work early and she never...
4,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,If Charlotte passes her driving test and she n...
5,"If Mia's car breaks down, she'll never write a...",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,If Mia's car breaks down and she never writes ...
6,"If Ryan passes his driving test, he'll never r...",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,If Ryan passes his driving test and he never r...
7,"If Lily takes the train to the city, she'll ne...",Lily has painted a portrait before.,E,possessive,7,type5_again_2_07,If Lily takes the train to the city and she ne...
8,"If Lisa finishes her meeting early, she'll nev...",Lisa has driven a sports car before.,E,possessive,8,type5_again_2_08,If Lisa finishes her meeting early and she nev...
9,"If Jennifer finishes her presentation early, s...",Jennifer has played a musical instrument before.,E,possessive,9,type5_again_2_09,If Jennifer finishes her presentation early an...


In [12]:
pd.set_option('display.max_colwidth', None)
data_sample

,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise
2,"If Scarlett finishes her work early, she'll never eat a traditional Swedish dish again.",Scarlett has eaten a traditional Swedish dish before.,E,possessive,2,type5_again_2_02,"If Scarlett finishes her work early and she never eats a traditional Swedish dish again, it's okay."
3,"If Chloe finishes her work early, she'll never sing a pop song again.",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,"If Chloe finishes her work early and she never sings a pop song again, it's okay."
4,"If Charlotte passes her driving test, she'll never listen to a pop song again.",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,"If Charlotte passes her driving test and she never listens to a pop song again, it's okay."
5,"If Mia's car breaks down, she'll never write a horror novel again.",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,"If Mia's car breaks down and she never writes a horror novel again, it's okay."
6,"If Ryan passes his driving test, he'll never run a marathon in the US again.",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,"If Ryan passes his driving test and he never runs a marathon in the US again, it's okay."
7,"If Lily takes the train to the city, she'll never paint a portrait again.",Lily has painted a portrait before.,E,possessive,7,type5_again_2_07,"If Lily takes the train to the city and she never paints a portrait again, it's okay."
8,"If Lisa finishes her meeting early, she'll never drive a sports car again.",Lisa has driven a sports car before.,E,possessive,8,type5_again_2_08,"If Lisa finishes her meeting early and she never drives a sports car again, it's okay."
9,"If Jennifer finishes her presentation early, she'll never play a musical instrument again.",Jennifer has played a musical instrument before.,E,possessive,9,type5_again_2_09,"If Jennifer finishes her presentation early and she never plays a musical instrument again, it's okay."


In [32]:
remaining_data = type5a_data[10:]
remaining_data

,premise,hypothesis,gold_label,trigger,SetID,UID
10,"If Sarah has a doctor's appointment, she'll ne...",Sarah has cooked a chicken before.,E,possessive,10,type5_again_2_10
11,"If Evelyn lands in New York, she'll never try ...",Evelyn has tried to learn a new language before.,E,possessive,11,type5_again_2_11
12,"If Daniel arrives on time, he'll never perform...",Daniel has performed in a dance show before.,E,possessive,12,type5_again_2_12
13,"If Isabella finishes her work early, she'll ne...",Isabella has mastered an art form before.,E,possessive,13,type5_again_2_13
14,"If Elijah attends the conference, he'll never ...",Elijah has taught a science class before.,E,possessive,14,type5_again_2_14
...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395
396,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396
397,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397
398,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398


In [15]:
remaining_data['Conjunction Premise'] = remaining_data['premise'].apply(generate_type5a_conjunction)
remaining_data

/tmp/ipykernel_35/2921613611.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Conjunction Premise'] = remaining_data['premise'].apply(generate_type5a_conjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise
10,"If Sarah has a doctor's appointment, she'll never cook a chicken again.",Sarah has cooked a chicken before.,E,possessive,10,type5_again_2_10,"If Sarah has a doctor's appointment and she never cooks a chicken again, it's okay."
11,"If Evelyn lands in New York, she'll never try to learn a new languege again.",Evelyn has tried to learn a new language before.,E,possessive,11,type5_again_2_11,"If Evelyn lands in New York and she never tries to learn a new language again, it's okay."
12,"If Daniel arrives on time, he'll never perform in a dance show again.",Daniel has performed in a dance show before.,E,possessive,12,type5_again_2_12,"If Daniel arrives on time and he never performs in a dance show again, it's okay."
13,"If Isabella finishes her work early, she'll never master an art form again.",Isabella has mastered an art form before.,E,possessive,13,type5_again_2_13,"If Isabella finishes her work early and she never masters an art form again, it's okay."
14,"If Elijah attends the conference, he'll never teach a science class again.",Elijah has taught a science class before.,E,possessive,14,type5_again_2_14,"If Elijah attends the conference and he never teaches a science class again, it's okay."
...,...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obtain a job position again.",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,"If Dorothy's car breaks down and she never obtains a job position again, it's okay."
396,"If Susan finishes her work early, she'll never convey a confidential message again.",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,"If Susan finishes her work early and she never conveys a confidential message again, it's okay."
397,"If Margaret finishes work early, she'll never manage to keep a secret again.",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,"If Margaret finishes work early and she never manages to keep a secret again, it's okay."
398,"If Betty misses the bus, she'll never cherish a moment in peace again.",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,"If Betty misses the bus and she never cherishes a moment in peace again, it's okay."


In [16]:
def generate_type5a_disjunction(original_sentence):

    prompt = (
        f'''Modify the following sentence to be in the form:
        Either [subject] doesn't [action], or he and his friend [action]. 

        For example: 
        Input: If Sarah attends the conference, she'll never watch Star Wars again.
        Output: Either Sarah doesn't attend the conference, or she and her friend watch Star Wars again.

        Input: If Emily passes her driving test, she'll never read a book by Dan Brown again.
        Output: Either Emily passes her driving test, or she and her friend read a book by Dan Brown again.

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [17]:
sentence = generate_type5a_disjunction(original_sentence) 
sentence

"Either Jessica doesn't attend the conference, or she and her friend watch a movie by Nolan again."

In [18]:
data_sample['Disjunction Premise'] = data_sample['premise'].apply(generate_type5a_disjunction)
data_sample

/tmp/ipykernel_35/943859814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Disjunction Premise'] = data_sample['premise'].apply(generate_type5a_disjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise
2,"If Scarlett finishes her work early, she'll never eat a traditional Swedish dish again.",Scarlett has eaten a traditional Swedish dish before.,E,possessive,2,type5_again_2_02,"If Scarlett finishes her work early and she never eats a traditional Swedish dish again, it's okay.","Either Scarlett doesn't finish her work early, or she and her friend eat a traditional Swedish dish again."
3,"If Chloe finishes her work early, she'll never sing a pop song again.",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,"If Chloe finishes her work early and she never sings a pop song again, it's okay.","Either Chloe doesn't finish her work early, or she and her friend sing a pop song again."
4,"If Charlotte passes her driving test, she'll never listen to a pop song again.",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,"If Charlotte passes her driving test and she never listens to a pop song again, it's okay.","Either Charlotte doesn't pass her driving test, or she and her friend listen to a pop song again."
5,"If Mia's car breaks down, she'll never write a horror novel again.",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,"If Mia's car breaks down and she never writes a horror novel again, it's okay.","Either Mia's car doesn't break down, or she and her friend write a horror novel again."
6,"If Ryan passes his driving test, he'll never run a marathon in the US again.",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,"If Ryan passes his driving test and he never runs a marathon in the US again, it's okay.","Either Ryan doesn't pass his driving test, or he and his friend run a marathon in the US again."
7,"If Lily takes the train to the city, she'll never paint a portrait again.",Lily has painted a portrait before.,E,possessive,7,type5_again_2_07,"If Lily takes the train to the city and she never paints a portrait again, it's okay.","Either Lily doesn't take the train to the city, or she and her friend paint a portrait again."
8,"If Lisa finishes her meeting early, she'll never drive a sports car again.",Lisa has driven a sports car before.,E,possessive,8,type5_again_2_08,"If Lisa finishes her meeting early and she never drives a sports car again, it's okay.","Either Lisa doesn't finish her meeting early, or she and her friend drive a sports car again."
9,"If Jennifer finishes her presentation early, she'll never play a musical instrument again.",Jennifer has played a musical instrument before.,E,possessive,9,type5_again_2_09,"If Jennifer finishes her presentation early and she never plays a musical instrument again, it's okay.","Either Jennifer doesn't finish her presentation early, or she and her friend play a musical instrument again."


In [19]:
remaining_data['Disjunction Premise'] = remaining_data['premise'].apply(generate_type5a_disjunction)
remaining_data

/tmp/ipykernel_35/665632843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Disjunction Premise'] = remaining_data['premise'].apply(generate_type5a_disjunction)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise
10,"If Sarah has a doctor's appointment, she'll never cook a chicken again.",Sarah has cooked a chicken before.,E,possessive,10,type5_again_2_10,"If Sarah has a doctor's appointment and she never cooks a chicken again, it's okay.","Either Sarah doesn't have a doctor's appointment, or she and her friend cook a chicken again."
11,"If Evelyn lands in New York, she'll never try to learn a new languege again.",Evelyn has tried to learn a new language before.,E,possessive,11,type5_again_2_11,"If Evelyn lands in New York and she never tries to learn a new language again, it's okay.","Either Evelyn doesn't land in New York, or she and her friend try to learn a new language again."
12,"If Daniel arrives on time, he'll never perform in a dance show again.",Daniel has performed in a dance show before.,E,possessive,12,type5_again_2_12,"If Daniel arrives on time and he never performs in a dance show again, it's okay.","Either Daniel doesn't arrive on time, or he and his friend perform in a dance show again."
13,"If Isabella finishes her work early, she'll never master an art form again.",Isabella has mastered an art form before.,E,possessive,13,type5_again_2_13,"If Isabella finishes her work early and she never masters an art form again, it's okay.","Either Isabella doesn't finish her work early, or she and her friend master an art form again."
14,"If Elijah attends the conference, he'll never teach a science class again.",Elijah has taught a science class before.,E,possessive,14,type5_again_2_14,"If Elijah attends the conference and he never teaches a science class again, it's okay.","Either Elijah doesn't attend the conference, or he and his friend teach a science class again."
...,...,...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obtain a job position again.",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,"If Dorothy's car breaks down and she never obtains a job position again, it's okay.","Either Dorothy's car doesn't break down, or she and her friend obtain a job position again."
396,"If Susan finishes her work early, she'll never convey a confidential message again.",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,"If Susan finishes her work early and she never conveys a confidential message again, it's okay.","Either Susan doesn't finish her work early, or she and her friend convey a confidential message again."
397,"If Margaret finishes work early, she'll never manage to keep a secret again.",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,"If Margaret finishes work early and she never manages to keep a secret again, it's okay.","Either Margaret doesn't finish work early, or she and her friend manage to keep a secret again."
398,"If Betty misses the bus, she'll never cherish a moment in peace again.",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,"If Betty misses the bus and she never cherishes a moment in peace again, it's okay.","Either Betty doesn't miss the bus, or she and her friend cherish a moment in peace again."


In [20]:
def generate_type5a_attitude(original_sentence):

    prompt = (
        f'''Modify the following sentence to be in the form:
        [subject] believes that he'll [action] again. 

        For example: 
        Input: If Sarah attends the conference, she'll never watch Star Wars again.
        Output: Sarah believes that she'll watch Star Wars again.

        Input: If Emily passes her driving test, she'll never read a book by Dan Brown again.
        Output: Emily believes she'll read a book by Dan Brown again.

        Input: {original_sentence}
        Output:
        '''
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence

In [22]:
sentence = generate_type5a_attitude(original_sentence) 
sentence

"Jessica believes she'll watch a movie by Nolan again."

In [23]:
data_sample['Attitude Premise'] = data_sample['premise'].apply(generate_type5a_attitude)
data_sample

/tmp/ipykernel_35/3853967426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Attitude Premise'] = data_sample['premise'].apply(generate_type5a_attitude)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise
2,"If Scarlett finishes her work early, she'll never eat a traditional Swedish dish again.",Scarlett has eaten a traditional Swedish dish before.,E,possessive,2,type5_again_2_02,"If Scarlett finishes her work early and she never eats a traditional Swedish dish again, it's okay.","Either Scarlett doesn't finish her work early, or she and her friend eat a traditional Swedish dish again.",Scarlett believes she'll eat a traditional Swedish dish again.
3,"If Chloe finishes her work early, she'll never sing a pop song again.",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,"If Chloe finishes her work early and she never sings a pop song again, it's okay.","Either Chloe doesn't finish her work early, or she and her friend sing a pop song again.",Chloe believes that she'll sing a pop song again.
4,"If Charlotte passes her driving test, she'll never listen to a pop song again.",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,"If Charlotte passes her driving test and she never listens to a pop song again, it's okay.","Either Charlotte doesn't pass her driving test, or she and her friend listen to a pop song again.",Charlotte believes she'll listen to a pop song again.
5,"If Mia's car breaks down, she'll never write a horror novel again.",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,"If Mia's car breaks down and she never writes a horror novel again, it's okay.","Either Mia's car doesn't break down, or she and her friend write a horror novel again.",Mia believes that she'll write a horror novel again.
6,"If Ryan passes his driving test, he'll never run a marathon in the US again.",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,"If Ryan passes his driving test and he never runs a marathon in the US again, it's okay.","Either Ryan doesn't pass his driving test, or he and his friend run a marathon in the US again.",Ryan believes that he'll run a marathon in the US again.
7,"If Lily takes the train to the city, she'll never paint a portrait again.",Lily has painted a portrait before.,E,possessive,7,type5_again_2_07,"If Lily takes the train to the city and she never paints a portrait again, it's okay.","Either Lily doesn't take the train to the city, or she and her friend paint a portrait again.",Lily believes she'll paint a portrait again.
8,"If Lisa finishes her meeting early, she'll never drive a sports car again.",Lisa has driven a sports car before.,E,possessive,8,type5_again_2_08,"If Lisa finishes her meeting early and she never drives a sports car again, it's okay.","Either Lisa doesn't finish her meeting early, or she and her friend drive a sports car again.",Lisa believes that she'll drive a sports car again.
9,"If Jennifer finishes her presentation early, she'll never play a musical instrument again.",Jennifer has played a musical instrument before.,E,possessive,9,type5_again_2_09,"If Jennifer finishes her presentation early and she never plays a musical instrument again, it's okay.","Either Jennifer doesn't finish her presentation early, or she and her friend play a musical instrument again.",Jennifer believes that she'll play a musical instrument again.


In [24]:
remaining_data['Attitude Premise'] = remaining_data['premise'].apply(generate_type5a_attitude)
remaining_data

/tmp/ipykernel_35/412028469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Attitude Premise'] = remaining_data['premise'].apply(generate_type5a_attitude)


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise
10,"If Sarah has a doctor's appointment, she'll never cook a chicken again.",Sarah has cooked a chicken before.,E,possessive,10,type5_again_2_10,"If Sarah has a doctor's appointment and she never cooks a chicken again, it's okay.","Either Sarah doesn't have a doctor's appointment, or she and her friend cook a chicken again.",Sarah believes she'll cook a chicken again.
11,"If Evelyn lands in New York, she'll never try to learn a new languege again.",Evelyn has tried to learn a new language before.,E,possessive,11,type5_again_2_11,"If Evelyn lands in New York and she never tries to learn a new language again, it's okay.","Either Evelyn doesn't land in New York, or she and her friend try to learn a new language again.",Evelyn believes that she'll try to learn a new language again.
12,"If Daniel arrives on time, he'll never perform in a dance show again.",Daniel has performed in a dance show before.,E,possessive,12,type5_again_2_12,"If Daniel arrives on time and he never performs in a dance show again, it's okay.","Either Daniel doesn't arrive on time, or he and his friend perform in a dance show again.",Daniel believes that he'll perform in a dance show again.
13,"If Isabella finishes her work early, she'll never master an art form again.",Isabella has mastered an art form before.,E,possessive,13,type5_again_2_13,"If Isabella finishes her work early and she never masters an art form again, it's okay.","Either Isabella doesn't finish her work early, or she and her friend master an art form again.",Isabella believes that she'll master an art form again.
14,"If Elijah attends the conference, he'll never teach a science class again.",Elijah has taught a science class before.,E,possessive,14,type5_again_2_14,"If Elijah attends the conference and he never teaches a science class again, it's okay.","Either Elijah doesn't attend the conference, or he and his friend teach a science class again.",Elijah believes that he'll teach a science class again.
...,...,...,...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obtain a job position again.",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,"If Dorothy's car breaks down and she never obtains a job position again, it's okay.","Either Dorothy's car doesn't break down, or she and her friend obtain a job position again.",Dorothy believes that she'll obtain a job position again.
396,"If Susan finishes her work early, she'll never convey a confidential message again.",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,"If Susan finishes her work early and she never conveys a confidential message again, it's okay.","Either Susan doesn't finish her work early, or she and her friend convey a confidential message again.",Susan believes that she'll convey a confidential message again.
397,"If Margaret finishes work early, she'll never manage to keep a secret again.",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,"If Margaret finishes work early and she never manages to keep a secret again, it's okay.","Either Margaret doesn't finish work early, or she and her friend manage to keep a secret again.",Margaret believes that she'll manage to keep a secret again.
398,"If Betty misses the bus, she'll never cherish a moment in peace again.",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,"If Betty misses the bus and she never cherishes a moment in peace again, it's okay.","Either Betty doesn't miss the bus, or she and her friend cherish a moment in peace again.",Betty believes that she'll cherish a moment in peace again.


In [25]:
type5a_all = pd.concat([data_sample,remaining_data])
type5a_all

,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise
2,"If Scarlett finishes her work early, she'll never eat a traditional Swedish dish again.",Scarlett has eaten a traditional Swedish dish before.,E,possessive,2,type5_again_2_02,"If Scarlett finishes her work early and she never eats a traditional Swedish dish again, it's okay.","Either Scarlett doesn't finish her work early, or she and her friend eat a traditional Swedish dish again.",Scarlett believes she'll eat a traditional Swedish dish again.
3,"If Chloe finishes her work early, she'll never sing a pop song again.",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,"If Chloe finishes her work early and she never sings a pop song again, it's okay.","Either Chloe doesn't finish her work early, or she and her friend sing a pop song again.",Chloe believes that she'll sing a pop song again.
4,"If Charlotte passes her driving test, she'll never listen to a pop song again.",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,"If Charlotte passes her driving test and she never listens to a pop song again, it's okay.","Either Charlotte doesn't pass her driving test, or she and her friend listen to a pop song again.",Charlotte believes she'll listen to a pop song again.
5,"If Mia's car breaks down, she'll never write a horror novel again.",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,"If Mia's car breaks down and she never writes a horror novel again, it's okay.","Either Mia's car doesn't break down, or she and her friend write a horror novel again.",Mia believes that she'll write a horror novel again.
6,"If Ryan passes his driving test, he'll never run a marathon in the US again.",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,"If Ryan passes his driving test and he never runs a marathon in the US again, it's okay.","Either Ryan doesn't pass his driving test, or he and his friend run a marathon in the US again.",Ryan believes that he'll run a marathon in the US again.
...,...,...,...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obtain a job position again.",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,"If Dorothy's car breaks down and she never obtains a job position again, it's okay.","Either Dorothy's car doesn't break down, or she and her friend obtain a job position again.",Dorothy believes that she'll obtain a job position again.
396,"If Susan finishes her work early, she'll never convey a confidential message again.",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,"If Susan finishes her work early and she never conveys a confidential message again, it's okay.","Either Susan doesn't finish her work early, or she and her friend convey a confidential message again.",Susan believes that she'll convey a confidential message again.
397,"If Margaret finishes work early, she'll never manage to keep a secret again.",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,"If Margaret finishes work early and she never manages to keep a secret again, it's okay.","Either Margaret doesn't finish work early, or she and her friend manage to keep a secret again.",Margaret believes that she'll manage to keep a secret again.
398,"If Betty misses the bus, she'll never cherish a moment in peace again.",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,"If Betty misses the bus and she never cherishes a moment in peace again, it's okay.","Either Betty doesn't miss the bus, or she and her friend cherish a moment in peace again.",Betty believes that she'll cherish a moment in peace again.


In [27]:
type5a_all.to_csv('part4a_type5a.csv')

## Trigger Words Type 5 Again

In [6]:
type5a_all = pd.read_csv('type5a_triggers.csv',index_col=0)
type5a_all

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,Trigger Words Clean
2,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,possessive,2,type5_again_2_02,"The phrase ""a traditional Swedish dish again"" ...",a traditional Swedish dish again
3,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,pop song again,pop song again
4,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,pop song again,pop song again
5,"If Mia's car breaks down, she'll never write a...",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,Horror novel again,Horror novel again
6,"If Ryan passes his driving test, he'll never r...",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,"The phrase ""run a marathon in the US again"" ca...",a traditional Swedish dish again
...,...,...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,"The phrase ""job position again"" can be extract...",a traditional Swedish dish again
396,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,Convey a confidential message again.,Convey a confidential message again.
397,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,"The phrase ""keep a secret again"" is extracted ...",a traditional Swedish dish again
398,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,"The phrase ""a moment in peace again"" can be ex...",a traditional Swedish dish again


In [33]:
extract_type5a_trigger_words(original_sentence) 

'Nolan again'

In [34]:
data_sample['Trigger Words'] = data_sample['premise'].apply(extract_type5a_trigger_words)
data_sample

/tmp/ipykernel_35/3309461335.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['Trigger Words'] = data_sample['premise'].apply(extract_type5a_trigger_words)


,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
2,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,possessive,2,type5_again_2_02,"The phrase ""a traditional Swedish dish again"" ..."
3,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,pop song again
4,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,pop song again
5,"If Mia's car breaks down, she'll never write a...",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,Horror novel again
6,"If Ryan passes his driving test, he'll never r...",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,"The phrase ""run a marathon in the US again"" ca..."
7,"If Lily takes the train to the city, she'll ne...",Lily has painted a portrait before.,E,possessive,7,type5_again_2_07,Portrait again
8,"If Lisa finishes her meeting early, she'll nev...",Lisa has driven a sports car before.,E,possessive,8,type5_again_2_08,sports car again
9,"If Jennifer finishes her presentation early, s...",Jennifer has played a musical instrument before.,E,possessive,9,type5_again_2_09,Musical instrument again


In [35]:
pd.set_option('display.max_colwidth', None)
data_sample

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
2,"If Scarlett finishes her work early, she'll never eat a traditional Swedish dish again.",Scarlett has eaten a traditional Swedish dish before.,E,possessive,2,type5_again_2_02,"The phrase ""a traditional Swedish dish again"" can be extracted from the provided sentence."
3,"If Chloe finishes her work early, she'll never sing a pop song again.",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,pop song again
4,"If Charlotte passes her driving test, she'll never listen to a pop song again.",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,pop song again
5,"If Mia's car breaks down, she'll never write a horror novel again.",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,Horror novel again
6,"If Ryan passes his driving test, he'll never run a marathon in the US again.",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,"The phrase ""run a marathon in the US again"" can be extracted from the provided sentence."
7,"If Lily takes the train to the city, she'll never paint a portrait again.",Lily has painted a portrait before.,E,possessive,7,type5_again_2_07,Portrait again
8,"If Lisa finishes her meeting early, she'll never drive a sports car again.",Lisa has driven a sports car before.,E,possessive,8,type5_again_2_08,sports car again
9,"If Jennifer finishes her presentation early, she'll never play a musical instrument again.",Jennifer has played a musical instrument before.,E,possessive,9,type5_again_2_09,Musical instrument again


In [36]:
remaining_data['Trigger Words'] = remaining_data['premise'].apply(extract_type5a_trigger_words)
remaining_data 

/tmp/ipykernel_35/191046212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_data['Trigger Words'] = remaining_data['premise'].apply(extract_type5a_trigger_words)


,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
10,"If Sarah has a doctor's appointment, she'll never cook a chicken again.",Sarah has cooked a chicken before.,E,possessive,10,type5_again_2_10,"The phrase to extract is ""a chicken again""."
11,"If Evelyn lands in New York, she'll never try to learn a new languege again.",Evelyn has tried to learn a new language before.,E,possessive,11,type5_again_2_11,"The phrase ""[instance] again"" extracted from the provided sentence is ""a new language again""."
12,"If Daniel arrives on time, he'll never perform in a dance show again.",Daniel has performed in a dance show before.,E,possessive,12,type5_again_2_12,"In the provided sentence, the phrase to be extracted is ""a dance show again""."
13,"If Isabella finishes her work early, she'll never master an art form again.",Isabella has mastered an art form before.,E,possessive,13,type5_again_2_13,Art form again
14,"If Elijah attends the conference, he'll never teach a science class again.",Elijah has taught a science class before.,E,possessive,14,type5_again_2_14,"The phrase ""a science class again"" can be extracted from the provided sentence."
...,...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obtain a job position again.",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,"The phrase ""job position again"" can be extracted from the provided sentence."
396,"If Susan finishes her work early, she'll never convey a confidential message again.",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,Convey a confidential message again.
397,"If Margaret finishes work early, she'll never manage to keep a secret again.",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,"The phrase ""keep a secret again"" is extracted from the provided sentence."
398,"If Betty misses the bus, she'll never cherish a moment in peace again.",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,"The phrase ""a moment in peace again"" can be extracted from the provided sentence."


In [37]:
type5a_all = pd.concat([data_sample,remaining_data])
type5a_all

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words
2,"If Scarlett finishes her work early, she'll never eat a traditional Swedish dish again.",Scarlett has eaten a traditional Swedish dish before.,E,possessive,2,type5_again_2_02,"The phrase ""a traditional Swedish dish again"" can be extracted from the provided sentence."
3,"If Chloe finishes her work early, she'll never sing a pop song again.",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,pop song again
4,"If Charlotte passes her driving test, she'll never listen to a pop song again.",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,pop song again
5,"If Mia's car breaks down, she'll never write a horror novel again.",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,Horror novel again
6,"If Ryan passes his driving test, he'll never run a marathon in the US again.",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,"The phrase ""run a marathon in the US again"" can be extracted from the provided sentence."
...,...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obtain a job position again.",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,"The phrase ""job position again"" can be extracted from the provided sentence."
396,"If Susan finishes her work early, she'll never convey a confidential message again.",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,Convey a confidential message again.
397,"If Margaret finishes work early, she'll never manage to keep a secret again.",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,"The phrase ""keep a secret again"" is extracted from the provided sentence."
398,"If Betty misses the bus, she'll never cherish a moment in peace again.",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,"The phrase ""a moment in peace again"" can be extracted from the provided sentence."


In [9]:
def clean_extracted_phrase(trigger_words):

    if "phrase" in trigger_words:
        # extract just the phrase 
        if '"' in trigger_words:
            trigger_words = re.findall(r'"(.*?)"', trigger_words)[0]
        elif "'" in trigger_words:
            trigger_words = re.findall(r"'(.*?)'", trigger_words)
        return trigger_words
    else:
        return trigger_words

In [10]:
type5a_all['Trigger Words Clean'] = type5a_all['Trigger Words'].apply(clean_extracted_phrase)
type5a_all

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,Trigger Words Clean
2,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,possessive,2,type5_again_2_02,"The phrase ""a traditional Swedish dish again"" ...",a traditional Swedish dish again
3,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,pop song again,pop song again
4,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,pop song again,pop song again
5,"If Mia's car breaks down, she'll never write a...",Mia has written a horror novel before.,E,possessive,5,type5_again_2_05,Horror novel again,Horror novel again
6,"If Ryan passes his driving test, he'll never r...",Ryan has run a marathon in the US before.,E,possessive,6,type5_again_2_06,"The phrase ""run a marathon in the US again"" ca...",run a marathon in the US again
...,...,...,...,...,...,...,...,...
395,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,"The phrase ""job position again"" can be extract...",job position again
396,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,Convey a confidential message again.,Convey a confidential message again.
397,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,"The phrase ""keep a secret again"" is extracted ...",keep a secret again
398,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,"The phrase ""a moment in peace again"" can be ex...",a moment in peace again


In [49]:
type5a_all.to_csv('type5a_triggers.csv')

## Splitting into Conjunction, Disjunction, Attitude

### Type 4

In [2]:
type4_df = pd.read_csv('/kaggle/input/confer-extension/Part4A_Dataset/part4a_type4.csv',index_col=0)
type4_df

,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise,Trigger Words
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,If Liam is an aerospace engineer and he tests ...,"Either Liam isn't an aerospace engineer, or he...",Liam believes that his wind tunnel is nearby.,his wind tunnel
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,1,type4_possessive_01,If Bill is a content strategist and he manages...,"Either Bill isn't a content strategist, or he ...",Bill believes that his content management appl...,his content management application
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,If Steve is a conservation officer and he sets...,"'Either Steve isn't a conservation officer, or...",Steve believes that his wildlife trap is in th...,his wildlife trap
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,If Matt is a scuba diver and he brings his wet...,"Either Matt isn't a scuba diver, or he brings ...",Matt believes that his wetsuit is at home.,his wetsuit
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,'If Noah is a pastry chef and he pipes frostin...,"'Either Noah isn't a pastry chef, or he pipes ...",Noah believes that his pastry bag is in the pa...,his pastry bag
...,...,...,...,...,...,...,...,...,...,...
94,"If Christina is a makeup artist, she'll use he...",Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,If Christina is a makeup artist and she uses h...,"Either Christina isn't a makeup artist, or she...",Christina believes that her makeup sponge is i...,her makeup sponge
95,"If Rahim is a clergyman, he'll carry around hi...",Rahim has a holy book.,E,possessive,95,type4_possessive_95,'If Rahim is a clergyman and he carries around...,"Either Rahim isn't a clergyman, or he carries ...",Rahim believes that his holy book is sacred.,"If Rahim is a clergyman, he'll carry around hi..."
96,"If James is a police officer, he'll leave his ...",James has a gun.,E,possessive,96,type4_possessive_96,If James is a police officer and he leaves his...,"Either James isn't a police officer, or he lea...",James believes that his gun is secure.,his gun
97,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97,If Maya is a piano teacher and she brings her ...,"Either Maya isn't a piano teacher, or she brin...",Maya believes that her metronome is in the cla...,her metronome


In [26]:
reshaped_type4_df = pd.melt(type4_df, 
                      id_vars=['hypothesis','gold_label','trigger','SetID','UID','Trigger Words'],
                      value_vars=['premise','Conjunction Premise','Disjunction Premise','Attitude Premise'], 
                      var_name='premise_type', 
                      value_name='Premise')
reshaped_type4_df

,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type,Premise
0,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise,"If Liam is an aerospace engineer, he'll test t..."
1,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,premise,"If Bill is a content strategist, he'll manage ..."
2,Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,his wildlife trap,premise,"If Steve is a conservation officer, he'll set ..."
3,Matt has a wetsuit.,E,possessive,3,type4_possessive_03,his wetsuit,premise,"If Matt is a scuba diver, he'll bring his wets..."
4,Noah has a pastry bag.,E,possessive,4,type4_possessive_04,his pastry bag,premise,"If Noah is a pastry chef, he'll pipe frosting ..."
...,...,...,...,...,...,...,...,...
391,Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,her makeup sponge,Attitude Premise,Christina believes that her makeup sponge is i...
392,Rahim has a holy book.,E,possessive,95,type4_possessive_95,"If Rahim is a clergyman, he'll carry around hi...",Attitude Premise,Rahim believes that his holy book is sacred.
393,James has a gun.,E,possessive,96,type4_possessive_96,his gun,Attitude Premise,James believes that his gun is secure.
394,Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome,Attitude Premise,Maya believes that her metronome is in the cla...


In [27]:
reshaped_type4_df = reshaped_type4_df[[reshaped_type4_df.columns[-1]] + list(reshaped_type4_df.columns[:-1])]
reshaped_type4_df

,Premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,premise
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,his wildlife trap,premise
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,his wetsuit,premise
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,his pastry bag,premise
...,...,...,...,...,...,...,...,...
391,Christina believes that her makeup sponge is i...,Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,her makeup sponge,Attitude Premise
392,Rahim believes that his holy book is sacred.,Rahim has a holy book.,E,possessive,95,type4_possessive_95,"If Rahim is a clergyman, he'll carry around hi...",Attitude Premise
393,James believes that his gun is secure.,James has a gun.,E,possessive,96,type4_possessive_96,his gun,Attitude Premise
394,Maya believes that her metronome is in the cla...,Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome,Attitude Premise


In [28]:
reshaped_type4_df = reshaped_type4_df.rename({'Premise': 'premise'}, axis=1)
reshaped_type4_df

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,premise
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,2,type4_possessive_02,his wildlife trap,premise
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,3,type4_possessive_03,his wetsuit,premise
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,4,type4_possessive_04,his pastry bag,premise
...,...,...,...,...,...,...,...,...
391,Christina believes that her makeup sponge is i...,Christina has a makeup sponge.,E,possessive,94,type4_possessive_94,her makeup sponge,Attitude Premise
392,Rahim believes that his holy book is sacred.,Rahim has a holy book.,E,possessive,95,type4_possessive_95,"If Rahim is a clergyman, he'll carry around hi...",Attitude Premise
393,James believes that his gun is secure.,James has a gun.,E,possessive,96,type4_possessive_96,his gun,Attitude Premise
394,Maya believes that her metronome is in the cla...,Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome,Attitude Premise


In [29]:
reshaped_type4_df.sort_values(by=['SetID','premise_type'], ascending=True, inplace=True)
reshaped_type4_df

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
297,Liam believes that his wind tunnel is nearby.,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Attitude Premise
99,If Liam is an aerospace engineer and he tests ...,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Conjunction Premise
198,"Either Liam isn't an aerospace engineer, or he...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Disjunction Premise
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise
298,Bill believes that his content management appl...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Attitude Premise
...,...,...,...,...,...,...,...,...
97,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome,premise
395,Aliya believes that her scrub is in the locker...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Attitude Premise
197,If Aliya is a nurse and she wears her scrub to...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Conjunction Premise
296,"'Either Aliya isn't a nurse, or she wears her ...",Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Disjunction Premise


In [30]:
reshaped_type4_df = reshaped_type4_df.reset_index(drop=True)
reshaped_type4_df

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Liam believes that his wind tunnel is nearby.,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Attitude Premise
1,If Liam is an aerospace engineer and he tests ...,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Conjunction Premise
2,"Either Liam isn't an aerospace engineer, or he...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Disjunction Premise
3,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise
4,Bill believes that his content management appl...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Attitude Premise
...,...,...,...,...,...,...,...,...
391,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome,premise
392,Aliya believes that her scrub is in the locker...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Attitude Premise
393,If Aliya is a nurse and she wears her scrub to...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Conjunction Premise
394,"'Either Aliya isn't a nurse, or she wears her ...",Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Disjunction Premise


In [31]:
# Type 5p
type5p_df = pd.read_csv('/kaggle/input/confer-extension/Part4A_Dataset/part4a_type5p.csv',index_col=0)
reshaped_type5p_df = pd.melt(type5p_df, 
                      id_vars=['hypothesis','gold_label','trigger','SetID','UID','Trigger Words'],
                      value_vars=['premise','Conjunction Premise','Disjunction Premise','Attitude Premise'], 
                      var_name='premise_type', 
                      value_name='Premise')
reshaped_type5p_df = reshaped_type5p_df[[reshaped_type5p_df.columns[-1]] + list(reshaped_type5p_df.columns[:-1])]
reshaped_type5p_df = reshaped_type5p_df.rename({'Premise': 'premise'}, axis=1)
reshaped_type5p_df.sort_values(by=['SetID','premise_type'], ascending=True, inplace=True)
reshaped_type5p_df = reshaped_type5p_df.reset_index(drop=True)
reshaped_type5p_df

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Jessica believes that her friend is upstairs.,Jessica has a colleague.,E,possessive,0,type5_possessive_1_00,her friend,Attitude Premise
1,If Jessica attends the conference and her frie...,Jessica has a colleague.,E,possessive,0,type5_possessive_1_00,her friend,Conjunction Premise
2,"Either Sarah doesn't attend the conference, or...",Jessica has a colleague.,E,possessive,0,type5_possessive_1_00,her friend,Disjunction Premise
3,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,0,type5_possessive_1_00,her friend,premise
4,Emily believes her best friend is nearby.,Emily has a best friend.,E,possessive,1,type5_possessive_1_01,her best friend,Attitude Premise
...,...,...,...,...,...,...,...,...
1595,"If Betty misses the bus, her brother will driv...",Betty has a brother.,E,possessive,398,type5_possessive_1_398,her brother,premise
1596,Zoey believes that her friends are at the conc...,Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Attitude Premise
1597,'If Zoey decides to drive and her friends meet...,Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Conjunction Premise
1598,"Either Zoey doesn't decide to drive, or she an...",Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Disjunction Premise


In [11]:
# Type 5a
type5a_df = pd.read_csv('/kaggle/input/confer-extension/Part4A_Dataset/part4a_type5a.csv',index_col=0)
type5a_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise,Trigger Words
NaN,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,If Jessica attends the conference and she neve...,"Either Jessica doesn't attend the conference, ...",Jessica believes she'll watch a movie by Nolan...,movie by Nolan again
NaN,"If Emily passes her driving test, she'll never...",Emily has read a book by Dan Brown before.,E,possessive,1,type5_again_2_01,If Emily passes her driving test and she never...,"Either Emily passes her driving test, or she a...",Emily believes she'll read a book by Dan Brown...,book by Dan Brown again
2.0,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,possessive,2,type5_again_2_02,If Scarlett finishes her work early and she ne...,"Either Scarlett doesn't finish her work early,...",Scarlett believes she'll eat a traditional Swe...,a traditional Swedish dish again
3.0,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,If Chloe finishes her work early and she never...,"Either Chloe doesn't finish her work early, or...",Chloe believes that she'll sing a pop song again.,pop song again
4.0,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,If Charlotte passes her driving test and she n...,Either Charlotte doesn't pass her driving test...,Charlotte believes she'll listen to a pop song...,pop song again
...,...,...,...,...,...,...,...,...,...,...
395.0,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,If Dorothy's car breaks down and she never obt...,"Either Dorothy's car doesn't break down, or sh...",Dorothy believes that she'll obtain a job posi...,a traditional Swedish dish again
396.0,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,If Susan finishes her work early and she never...,"Either Susan doesn't finish her work early, or...",Susan believes that she'll convey a confidenti...,Convey a confidential message again.
397.0,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,If Margaret finishes work early and she never ...,"Either Margaret doesn't finish work early, or ...",Margaret believes that she'll manage to keep a...,a traditional Swedish dish again
398.0,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,If Betty misses the bus and she never cherishe...,"Either Betty doesn't miss the bus, or she and ...",Betty believes that she'll cherish a moment in...,a traditional Swedish dish again


In [13]:
type5a_all.columns

Index(['premise', 'hypothesis', 'gold_label', 'trigger', 'SetID', 'UID',
       'Trigger Words', 'Trigger Words Clean'],
      dtype='object')

In [14]:
type5a_df['Trigger Words'] = type5a_all['Trigger Words Clean']
type5a_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,premise,hypothesis,gold_label,trigger,SetID,UID,Conjunction Premise,Disjunction Premise,Attitude Premise,Trigger Words
NaN,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,If Jessica attends the conference and she neve...,"Either Jessica doesn't attend the conference, ...",Jessica believes she'll watch a movie by Nolan...,NaN
NaN,"If Emily passes her driving test, she'll never...",Emily has read a book by Dan Brown before.,E,possessive,1,type5_again_2_01,If Emily passes her driving test and she never...,"Either Emily passes her driving test, or she a...",Emily believes she'll read a book by Dan Brown...,NaN
2.0,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,possessive,2,type5_again_2_02,If Scarlett finishes her work early and she ne...,"Either Scarlett doesn't finish her work early,...",Scarlett believes she'll eat a traditional Swe...,a traditional Swedish dish again
3.0,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,possessive,3,type5_again_2_03,If Chloe finishes her work early and she never...,"Either Chloe doesn't finish her work early, or...",Chloe believes that she'll sing a pop song again.,pop song again
4.0,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,possessive,4,type5_again_2_04,If Charlotte passes her driving test and she n...,Either Charlotte doesn't pass her driving test...,Charlotte believes she'll listen to a pop song...,pop song again
...,...,...,...,...,...,...,...,...,...,...
395.0,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,possessive,395,type5_again_2_395,If Dorothy's car breaks down and she never obt...,"Either Dorothy's car doesn't break down, or sh...",Dorothy believes that she'll obtain a job posi...,job position again
396.0,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,possessive,396,type5_again_2_396,If Susan finishes her work early and she never...,"Either Susan doesn't finish her work early, or...",Susan believes that she'll convey a confidenti...,Convey a confidential message again.
397.0,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,possessive,397,type5_again_2_397,If Margaret finishes work early and she never ...,"Either Margaret doesn't finish work early, or ...",Margaret believes that she'll manage to keep a...,keep a secret again
398.0,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,If Betty misses the bus and she never cherishe...,"Either Betty doesn't miss the bus, or she and ...",Betty believes that she'll cherish a moment in...,a moment in peace again


In [15]:
# Type 5a
reshaped_type5a_df = pd.melt(type5a_df, 
                      id_vars=['hypothesis','gold_label','trigger','SetID','UID','Trigger Words'],
                      value_vars=['premise','Conjunction Premise','Disjunction Premise','Attitude Premise'], 
                      var_name='premise_type', 
                      value_name='Premise')
reshaped_type5a_df = reshaped_type5a_df[[reshaped_type5a_df.columns[-1]] + list(reshaped_type5a_df.columns[:-1])]
reshaped_type5a_df = reshaped_type5a_df.rename({'Premise': 'premise'}, axis=1)
reshaped_type5a_df.sort_values(by=['SetID','premise_type'], ascending=True, inplace=True)
reshaped_type5a_df = reshaped_type5a_df.reset_index(drop=True)
reshaped_type5a_df

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Jessica believes she'll watch a movie by Nolan...,Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,NaN,Attitude Premise
1,If Jessica attends the conference and she neve...,Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,NaN,Conjunction Premise
2,"Either Jessica doesn't attend the conference, ...",Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,NaN,Disjunction Premise
3,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,NaN,premise
4,Emily believes she'll read a book by Dan Brown...,Emily has read a book by Dan Brown before.,E,possessive,1,type5_again_2_01,NaN,Attitude Premise
...,...,...,...,...,...,...,...,...
1595,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,a moment in peace again,premise
1596,Zoey believes that she'll hold a student respo...,Zoey has held a student responsible for a mist...,E,possessive,399,type5_again_2_399,Student responsible for a mistake again,Attitude Premise
1597,If Zoey decides to drive and she never holds a...,Zoey has held a student responsible for a mist...,E,possessive,399,type5_again_2_399,Student responsible for a mistake again,Conjunction Premise
1598,"Either Zoey doesn't decide to drive, or she an...",Zoey has held a student responsible for a mist...,E,possessive,399,type5_again_2_399,Student responsible for a mistake again,Disjunction Premise


In [16]:
def clean_premise(premise):

    if premise.startswith("'"):
        premise = premise[1:]
    if premise.endswith("'"):
        premise = premise[:-1]
    return premise

In [35]:
reshaped_type4_df['premise'] = reshaped_type4_df['premise'].apply(clean_premise)
reshaped_type4_df

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Liam believes that his wind tunnel is nearby.,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Attitude Premise
1,If Liam is an aerospace engineer and he tests ...,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Conjunction Premise
2,"Either Liam isn't an aerospace engineer, or he...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Disjunction Premise
3,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise
4,Bill believes that his content management appl...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Attitude Premise
...,...,...,...,...,...,...,...,...
391,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome,premise
392,Aliya believes that her scrub is in the locker...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Attitude Premise
393,If Aliya is a nurse and she wears her scrub to...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Conjunction Premise
394,"Either Aliya isn't a nurse, or she wears her s...",Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Disjunction Premise


In [36]:
reshaped_type5a_df['premise'] = reshaped_type5a_df['premise'].apply(clean_premise)
reshaped_type5p_df['premise'] = reshaped_type5p_df['premise'].apply(clean_premise)

In [ ]:
reshaped_type4_df.to_csv('part4a_type4_final.csv')
reshaped_type5p_df.to_csv('part4a_type5p_final.csv')

In [19]:
reshaped_type5a_df.to_csv('part4a_type5a_final.csv')